In [ ]:
# Define a flag to check if %cd .. has been run
if 'cd_executed' not in globals():
    %cd ..
    cd_executed = True  # Set the flag to True after the command runs


In [ ]:
from liegroups.numpy import SO3 as SO3_np
import tqdm

from __future__ import absolute_import, division, print_function

import numpy as np
import time
import pdb
import shutil

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from tensorboardX import SummaryWriter

import json

from utils import *
from kitti_utils import *
from layers import *

import datasets
# import networks
from networks.velo_decoder import VeloDecoder
from networks.gravity_decoder import GravityDecoder
from networks.depth_decoder import DepthDecoder
from networks.resnet_encoder import ResnetEncoder
from ekf import EKFModel
from ekf import proc_vis_covar

from options import MonodepthOptions


import matplotlib.pyplot as plt

import matplotlib.animation as animation


# Metrics
import importlib
import cv2
import datasets.midair



In [ ]:
class BackprojectDepth(nn.Module):
    """Layer to transform a depth image into a point cloud
    """
    def __init__(self, batch_size, height, width):
        super(BackprojectDepth, self).__init__()

        self.batch_size = batch_size
        self.height = height
        self.width = width

        meshgrid = np.meshgrid(range(self.width), range(self.height), indexing='xy')
        self.id_coords = np.stack(meshgrid, axis=0).astype(np.float32)
        self.id_coords = nn.Parameter(torch.from_numpy(self.id_coords),
                                      requires_grad=False)

        self.ones = nn.Parameter(torch.ones(self.batch_size, 1, self.height * self.width),
                                 requires_grad=False)

        self.pix_coords = torch.unsqueeze(torch.stack(
            [self.id_coords[0].view(-1), self.id_coords[1].view(-1)], 0), 0)
        self.pix_coords = self.pix_coords.repeat(batch_size, 1, 1)
        self.pix_coords = nn.Parameter(torch.cat([self.pix_coords, self.ones], 1),
                                       requires_grad=False)

    def forward(self, depth, inv_K):
        cam_points = torch.matmul(inv_K[:, :3, :3], self.pix_coords)

        cam_points = depth.view(self.batch_size, 1, -1) * cam_points
        cam_points = torch.cat([cam_points, self.ones], 1)

        return cam_points


class Project3D(nn.Module):
    """Layer which projects 3D points into a camera with intrinsics K and at position T
    """
    def __init__(self, batch_size, height, width, eps=1e-7):
        super(Project3D, self).__init__()

        self.batch_size = batch_size
        self.height = height
        self.width = width
        self.eps = eps

    def forward(self, points, K, T):
        P = torch.matmul(K, T)[:, :3, :]

        cam_points = torch.matmul(P, points)

        pix_coords = cam_points[:, :2, :] / (cam_points[:, 2, :].unsqueeze(1) + self.eps)
        pix_coords = pix_coords.view(self.batch_size, 2, self.height, self.width)
        pix_coords = pix_coords.permute(0, 2, 3, 1)
        pix_coords[..., 0] /= self.width - 1
        pix_coords[..., 1] /= self.height - 1
        pix_coords = (pix_coords - 0.5) * 2
        return pix_coords


In [ ]:
def compute_imu_pose_with_inv(alpha, R_c, R_cbt_bc, delta_t, gravities, velocities, trans_scale_factor):
    """
    -> Rotation is directly obtained using preintegrated q
    -> Translation is obtained by preintegrated alpha and q
    -> Return:
        -> poses: [from 0 to -1, from 0 to 1]
        -> poses_inv: [from -1 to 0, from 1 to 0]
    """
    # NOTE: T_c denotes [from 0 to -1, from 1 to 0]
    # NOTE: T_c_inv denotes [from -1 to 0, from 0 to 1]
    T_c, T_c_inv = [], []
    for i in [0]:
        rot = R_c[i] # [B, 3, 3]
        dt = delta_t[i].unsqueeze(-1).unsqueeze(-1) # [B, 1, 1]
        trans = alpha[i].unsqueeze(-1) + R_c[i] @ R_cbt_bc[i].unsqueeze(-1) - R_cbt_bc[i].unsqueeze(-1) - 0.5 * gravities[i].unsqueeze(-1) * dt * dt + velocities[i].unsqueeze(-1) * dt # [B, 3, 1]
        
        # NOTE: trans is re-scaled by 5.4, but gravities and velocities are still the original scale
        trans = trans / trans_scale_factor
                
        T_mat = torch.cat([rot, trans], dim=2) # [B, 3, 4]
        fill = T_mat.new_zeros([T_mat.shape[0], 1, 4]) # [B, 1, 4]
        fill[:, :, -1] = 1
        T_mat = torch.cat([T_mat, fill], dim=1) # [B, 4, 4]
        T_c.append(T_mat) # [B, 4, 4]
        T_c_inv.append(T_mat.inverse()) # [B, 4, 4]
    
    # NOTE: the indices in poses/poses_inv are different from T_c/T_c_inv
    # -> poses: [from 0 to -1, from 0 to 1]
    # -> poses_inv: [from -1 to 0, from 1 to 0]
    # poses = [T_c_inv[0],T_mat[0]]
    poses = [T_c[0], T_c_inv[0]]
    return poses


In [ ]:
def custom_collate(batch):
    # remove empty items
    batch = [item for item in batch if item !={}]
    return torch.utils.data.default_collate(batch)

In [ ]:
def custom_sort_key(item):
    # Extract components from the filepath
    parts = item.split('/')
    kite_training = parts[0]  # 'Kite_training'
    weather = parts[1]        # e.g., 'cloudy'
    sensor = parts[2]         # e.g., 'sensor'
    trajectory_part = parts[3]  # e.g., 'trajectory_3000 397 l'

    # Split the trajectory part
    trajectory_str, number_str, *rest = trajectory_part.split(' ')

    # Extract the trajectory number and convert to integer
    trajectory_num = int(trajectory_str.replace('trajectory_', ''))

    # Check if number_str contains a range (e.g., '397-400')
    if '-' in number_str:
        # Extract the range numbers
        range_numbers = list(map(int, number_str.split('-')))
        number = range_numbers[0]  # Use the start of the range for sorting
    else:
        number = int(number_str)

    # Return a tuple as the sort key in the desired order
    return (kite_training, weather, sensor, trajectory_num, number)


In [ ]:
print(os.path.join("splits", "eigen_midair", "{}_files.txt"))
fpath = os.path.join("splits","eigen_midair", "{}_files.txt")

train_filenames = readlines(fpath.format("test"))

train_filenames = sorted(train_filenames, key=custom_sort_key)

In [ ]:
dataset = datasets.midair.MidAirDataset

fpath = os.path.join("splits","eigen_midair", "{}_files.txt")
train_filenames = readlines(fpath.format("test"))

device = torch.device("cpu")

### Load models

In [ ]:
models = {}
models["velo_encoder"] = ResnetEncoder(
    18,
    "pretrained",
    num_input_images=2)

models["velo"] = VeloDecoder(
    models["velo_encoder"].num_ch_enc,
    num_input_features=1,
    num_frames_to_predict_for=1)


# Initialize gravity networks
models["gravity_encoder"] = ResnetEncoder(
    18,
    "pretrained",
    num_input_images=2)


models["gravity"] = GravityDecoder(
    models["gravity_encoder"].num_ch_enc,
    num_input_features=1,
    num_frames_to_predict_for=1)


models["encoder"] = ResnetEncoder(
            50,
            False)


models["depth"] = DepthDecoder(
    models["encoder"].num_ch_enc,scales=range(1))




In [ ]:
load_weights_folder = "/Users/ibrahimhassan/Documents/Documents/thesis_DynaDepth/tmp_training_use_imu_True_use_ekf_True/dynadepth_20241014/models/weights_29"
vel_encoder_path = os.path.join(load_weights_folder, "velo_encoder.pth")
vel_path = os.path.join(load_weights_folder, "velo.pth")

grav_encoder_path = os.path.join(load_weights_folder, "gravity_encoder.pth")
grav_path = os.path.join(load_weights_folder, "gravity.pth")

encoder_path = os.path.join(load_weights_folder, "encoder.pth")
decoder_path = os.path.join(load_weights_folder, "depth.pth")

In [ ]:
models["velo_encoder"].load_state_dict(torch.load(vel_encoder_path,map_location=torch.device('cpu')))
models["velo"].load_state_dict(torch.load(vel_path,map_location=torch.device('cpu')))
models["gravity_encoder"].load_state_dict(torch.load(grav_encoder_path,map_location=torch.device('cpu')))
models["gravity"].load_state_dict(torch.load(grav_path,map_location=torch.device('cpu')))

encoder_dict = torch.load(encoder_path,map_location=torch.device('cpu'))
models["encoder"].load_state_dict({k: v for k, v in encoder_dict.items() if k in models["encoder"].state_dict()})
models["depth"].load_state_dict(torch.load(decoder_path,map_location=torch.device('cpu')))
models["encoder"].eval();
models["depth"].eval();

### Helper functions

In [ ]:
def invert_pose(pose):
    
    pose_red = torch.squeeze(pose)
    # R = torch.squeeze(pose[:3, :3])
    # t = torch.squeeze(pose[:3, 3:])
    
    R = pose_red[:3, :3]
    t = pose_red[:3, 3:]
    R_inv = R.transpose(0, 1)
    t_inv = -torch.matmul(R_inv, t)
    pose_inv = torch.eye(4).to(pose.device)
    pose_inv[:3, :3] = R_inv
    pose_inv[:3, 3:] = t_inv
    return pose_inv


In [ ]:
def get_poses_depth(data_load, models, alpha, R_c, R_cbt_bc, delta_t, TRANS_SCALE_FACTOR):
    IMU_sequence = []
    Images_sequence = []
    list_poses = []
    for idx, data in enumerate(data_load):


        for key, ipt in data.items():
            if key in [("preint_imu", -1, 0), ("preint_imu", 0, 1)]:
                for pkey, pipt in data[key].items():
                    data[key][pkey] = pipt.to(device, dtype=torch.float32)
                    if pkey not in ["v_norm", "delta_t"]:
                        pipt = pipt.unsqueeze(0)
                        data[key][pkey] = pipt.to(device, dtype=torch.float32)
                    else:
                        data[key][pkey] = pipt.to(device, dtype=torch.float32)
            else:
                ipt = ipt.unsqueeze(0)
                data[key] = ipt.to(device, dtype=torch.float32)



        IMU = data[('preint_imu', 0, 1)]
        IMU_sequence.append(IMU)
        Images_sequence.append(data[('color', 0, 0)])

        # Get IMU POSES
        pair_inputs = [data[('color', 0, 0)], data[('color', 1, 0)]]

        velo_inputs = [models["velo_encoder"](torch.cat(pair_inputs, 1))]
        gravity_inputs = [models["gravity_encoder"](torch.cat(pair_inputs, 1))]

        velocity = models["velo"](velo_inputs)
        gravity = models["gravity"](gravity_inputs)

        poses = compute_imu_pose_with_inv(alpha, R_c, R_cbt_bc, delta_t, gravity, velocity, TRANS_SCALE_FACTOR)
        list_poses.append(poses[0])

    return list_poses

In [ ]:
def get_depth_trajectory(batch_size, init_image, depth_init, list_poses, data):

    depth_after_imu = [depth_init]
    list_images_imu = [init_image]
    depth_loop = depth_init

    for i in range(len(list_poses) - 1):
        inv_pose = invert_pose(list_poses[i])

        DEPTH_POINT_CLOUDS = backproject_depth(depth_loop, data[("inv_K", 0)])
        P = torch.matmul(data[("K", 0)], inv_pose)[:, :3, :]
        TRANSFORMED_DEPTH_POINT_CLOUDS_FOR_Z = torch.matmul(P, DEPTH_POINT_CLOUDS)

        # Update depth
        new_depth = TRANSFORMED_DEPTH_POINT_CLOUDS_FOR_Z[:, 2, :].view(batch_size, 1, height, width)

        P = torch.matmul(data[("K", 0)], list_poses[i])[:, :3, :]
        TRANSFORMED_DEPTH_POINT_CLOUDS = torch.matmul(P, DEPTH_POINT_CLOUDS)

        pix_coords_depth = TRANSFORMED_DEPTH_POINT_CLOUDS[:, :2, :] / (TRANSFORMED_DEPTH_POINT_CLOUDS[:, 2, :].unsqueeze(1) + 0.001)
        pix_coords_depth = pix_coords_depth.view(batch_size, 2, height, width)
        pix_coords_depth = pix_coords_depth.permute(0, 2, 3, 1)
        pix_coords_depth[..., 0] /= width - 1
        pix_coords_depth[..., 1] /= height - 1
        pix_coords_depth = (pix_coords_depth - 0.5) * 2

        depth_loop = F.grid_sample(new_depth, pix_coords_depth, padding_mode="border")
        depth_after_imu.append(depth_loop)

        # Warp the initial image to new pose
        pix_coords_image = project_3d(DEPTH_POINT_CLOUDS, data[("K", 0)], list_poses[i])
        new_transformed_image = F.grid_sample(list_images_imu[i], pix_coords_image, padding_mode="border")
        list_images_imu.append(new_transformed_image)

    return depth_after_imu, depth_init, list_images_imu


In [ ]:
def get_video_depth(depth_after_imu, filename):

    # Example data (you can replace this with your actual frames)
    frames = depth_after_imu  # List of 50 random frames

    # Create a figure and axis
    fig, ax = plt.subplots()

    # Create an initial frame
    frame_data = frames[0]
    # print(frame_data[0].permute(1,2,0).detach().numpy())
    # im = ax.imshow(frame_data[0].permute(1,2,0).detach().numpy(), cmap='viridis')
    im = ax.imshow(frame_data[0].permute(1,2,0).detach().numpy(), cmap='viridis')

    # Update function for animation
    def update(frame):
        # print(frame[0].permute(1,2,0).detach().numpy().shape)
        # im.set_data(frame[0].permute(1,2,0).detach().numpy()
        im.set_data(frame[0].permute(1,2,0).detach().numpy())
        return [im]

    # Create an animation
    ani = animation.FuncAnimation(
        fig, update, frames=frames, interval=100, blit=True
    )

    # Save the animation as a video file (e.g., MP4)
    ani.save(filename, writer='ffmpeg', fps=30)

    # To display the animation in a notebook (optional)
    # from IPython.display import HTML
    # HTML(ani.to_jshtml())

    plt.show()

## Trajectory 1: 

In [ ]:
# Main code
# INIT DEPTH PREDICTION
min_depth = 0.1
max_depth = 80

height = 512
width = 512
batch_size = 1

backproject_depth = BackprojectDepth(batch_size, height, width)
project_3d = Project3D(batch_size, height, width)
fraction_trajectory_1 = train_filenames[1633:1633+100]

dataset = datasets.midair.MidAirDataset
fpath = os.path.join("splits", "eigen_midair", "{}_files.txt")
data_path = "MidAir"

print("Creating dataset trajectory 1")
dataset_trajectory_1 = dataset(
    data_path, fraction_trajectory_1, 
    height, width,
    [0, -1, 1],
    select_file_names=True,
    num_scales=1, 
    use_imu=True,
    k_imu_clip=5, is_train=False, img_ext='.jpg' 
)   

In [ ]:
def compute_errors(gt, pred):
    """Compute error metrics between predicted and ground truth depths."""
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25     ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse = np.sqrt(((gt - pred) ** 2).mean())
    rmse_log = np.sqrt(((np.log(gt) - np.log(pred)) ** 2).mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)
    sq_rel = np.mean(((gt - pred) ** 2) / gt)

    return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3


In [ ]:
# Initialize lists to store data
dataloader_trajectory_1 = DataLoader(dataset_trajectory_1, 1, shuffle=False, num_workers=0, pin_memory=True, drop_last=False)
list_frames_traj_1 = []
list_gt_frames_traj_1 = []
depth_gt_list = []  # List to store ground truth depth maps
alpha, R_c, R_cbt_bc, delta_t, gravities, velocities = [], [], [], [], [], []

avg_velocity = []

for idx, sample in enumerate(dataset_trajectory_1):
    # Standard processing
    for key, ipt in sample.items():
        if key in [("preint_imu", -1, 0), ("preint_imu", 0, 1)]:
            for pkey, pipt in sample[key].items():
                sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                if pkey not in ["v_norm", "delta_t"]:
                    pipt = pipt.unsqueeze(0)
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                else:
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
        else:
            ipt = ipt.unsqueeze(0)
            sample[key] = ipt.to(device, dtype=torch.float32)

    if idx == 0:
        init_sample = sample
        init_image = sample[('color', 0, 0)]
        disp_gt_batch = sample["disp_gt"]
        depth_gt_init = (height // 2) / disp_gt_batch.to(torch.float32)
        depth_gt_init = torch.clamp(depth_gt_init, min_depth, max_depth)
        depth_gt_init = depth_gt_init[:, 0].detach()

    frames = sample[('color', 0, 0)]
    disp_gt = sample["disp_gt"]
    disp_gt = (height // 2) / disp_gt.to(torch.float32)
    disp_gt = torch.clamp(disp_gt, min_depth, max_depth)
    depth_gt_sample = disp_gt[:, 0].detach()

    # Store ground truth depth maps
    depth_gt_list.append(depth_gt_sample.cpu())

    # Collect frames for visualization
    fig, (ax1, ax2) = plt.subplots(figsize=(10, 10), nrows=1, ncols=2)
    frame_data = frames
    im_1 = ax1.imshow(frame_data[0].permute(1, 2, 0).cpu().detach().numpy())
    im_2 = ax2.imshow(depth_gt_sample.permute(1, 2, 0).cpu().numpy(), cmap='viridis_r')
    list_frames_traj_1.append(im_1)
    list_gt_frames_traj_1.append(im_2)
    plt.close(fig)

    # Get IMU parameters
    next_imu = sample[('preint_imu', -1, 0)]
    alpha.append(next_imu["alpha"])
    R_c.append(next_imu["R_c"])
    R_cbt_bc.append(next_imu["R_cbt_bc"])
    delta_t.append(next_imu["delta_t"])
    avg_velocity.append(next_imu["v_norm"])

# Compute average velocity
vel_1 = torch.mean(torch.stack(avg_velocity), dim=0)
print(f"Average velocity: {vel_1}")

# Add code to create video of the original trajectory
image_frames_traj_1 = [im.get_array() for im in list_frames_traj_1]
gt_frames_traj_1 = [im.get_array() for im in list_gt_frames_traj_1]

# Create a figure and axis for the animation
fig, ax = plt.subplots()
# Initialize the image on the axis (using the first frame's shape)
im = ax.imshow(image_frames_traj_1[0], cmap='viridis')

# Update function for animation
def update(frame):
    im.set_data(frame)
    return [im]

# Create an animation
ani = animation.FuncAnimation(
    fig, update, frames=image_frames_traj_1, interval=100, blit=True
)

# Save the animation as a video file (e.g., MP4)
ani.save('Trajectories_anim/trajectory_1.mp4', writer='ffmpeg', fps=25)
plt.close(fig)

# Depth prediction using the same method as the second file
TRANS_SCALE_FACTOR = 5.4
pair_inputs = [init_sample[('color', 0, 0)], init_sample[('color', 1, 0)]]

# Generate velocity and gravity inputs
velo_inputs = [models["velo_encoder"](torch.cat(pair_inputs, 1))]
gravity_inputs = [models["gravity_encoder"](torch.cat(pair_inputs, 1))]

velocity = models["velo"](velo_inputs)
gravity = models["gravity"](gravity_inputs)

R_cbt_bc[0] = R_cbt_bc[0]

# Get depth prediction
with torch.no_grad():
    output = models["depth"](models["encoder"](init_image))
    disp_trajectory_1 = output

# Adjusted depth prediction
pred_disp, _ = disp_to_depth(disp_trajectory_1[("disp", 0)], min_depth, max_depth)
pred_disp = pred_disp.cpu()[:, 0]
pred_depth = 1 / pred_disp

# Apply median scaling
ratio = np.median(depth_gt_init.cpu().numpy()) / np.median(pred_depth.cpu().numpy())
pred_depth *= ratio
pred_depth = pred_depth.to(init_image.device).unsqueeze(0)

# Compute errors between predicted and ground truth depths
##########################################################
pred_depth_tmp = pred_depth.squeeze()
pred_depth_tmp = pred_depth_tmp.detach().cpu().numpy()
depth_gt_init_tmp = depth_gt_init.squeeze()
depth_gt_init_tmp = depth_gt_init_tmp.detach().cpu().numpy()

mask = (depth_gt_init_tmp > min_depth) & (depth_gt_init_tmp < max_depth)
pred_depth_flat = pred_depth_tmp[mask]

gt_depth_flat = depth_gt_init_tmp[mask]
print(f"init error : {compute_errors(gt_depth_flat, pred_depth_flat)[2]}")
##########################################################

# Generate poses and depth trajectory
poses_depth = get_poses_depth(dataset_trajectory_1, models, alpha, R_c, R_cbt_bc, delta_t, TRANS_SCALE_FACTOR)
depth_after_imu, depth_init_trajectory_1, images_after_imu = get_depth_trajectory(
    batch_size, init_image, pred_depth, poses_depth, init_sample
)

# Save depth video
get_video_depth(depth_after_imu, "Depth_anim/depth_trajectory_1.mp4")

# Function to compute errors
def compute_errors(gt, pred):
    """Compute error metrics between predicted and ground truth depths."""
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25     ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse = np.sqrt(((gt - pred) ** 2).mean())
    rmse_log = np.sqrt(((np.log(gt) - np.log(pred)) ** 2).mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)
    sq_rel = np.mean(((gt - pred) ** 2) / gt)

    return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3

# Now, compute errors between the predicted depth maps and ground truth
# Collect predicted depth maps from depth_after_imu
predicted_depth_maps = [depth.cpu().squeeze(0) for depth in depth_after_imu]

errors_trajectory_1 = []

for pred_depth_map, gt_depth_map in zip(predicted_depth_maps, depth_gt_list):
    pred_depth_map_np = pred_depth_map.detach().numpy() 
    gt_depth_map_np = gt_depth_map.numpy()
    
    # Apply mask to valid depth values
    mask = (gt_depth_map_np > min_depth) & (gt_depth_map_np < max_depth)
    
    pred_depth_flat = pred_depth_map_np[mask]
    gt_depth_flat = gt_depth_map_np[mask]
    
    # Ensure predicted depths are within valid range
    pred_depth_flat = np.clip(pred_depth_flat, min_depth, max_depth)
    
    # Compute error metrics
    errors_trajectory_1.append(compute_errors(gt_depth_flat, pred_depth_flat))

# Compute mean errors
mean_errors = np.mean(errors_trajectory_1, axis=0)

print("\n  " + ("{:>8} | " * 7).format("abs_rel", "sq_rel", "rmse", "rmse_log", "a1", "a2", "a3"))
print(("{: 8.3f}  " * 7).format(*mean_errors))


In [ ]:
rmse_trajectory_1 = [item[2] for item in errors_trajectory_1]
plt.title("RMSE as prediction horizon increases")
plt.plot(rmse_trajectory_1)
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")

#### Trajectory 2

In [ ]:
# Main code
# INIT DEPTH PREDICTION
min_depth = 0.1
max_depth = 80

height = 512
width = 512
batch_size = 1

backproject_depth = BackprojectDepth(batch_size, height, width)
project_3d = Project3D(batch_size, height, width)
fraction_trajectory_2 = train_filenames[1733:1733+100]

dataset = datasets.midair.MidAirDataset
fpath = os.path.join("splits", "eigen_midair", "{}_files.txt")
data_path = "MidAir"

print("Creating dataset trajectory 1")
dataset_trajectory_2 = dataset(
    data_path, fraction_trajectory_2, 
    height, width,
    [0, -1, 1],
    select_file_names=True,
    num_scales=1, 
    use_imu=True,
    k_imu_clip=5, is_train=False, img_ext='.jpg' 
)   

In [ ]:
# Initialize lists to store data
dataloader_trajectory_2 = DataLoader(dataset_trajectory_2, 1, shuffle=False, num_workers=0, pin_memory=True, drop_last=False)
list_frames_traj_2 = []
list_gt_frames_traj_2 = []
depth_gt_list = []  # List to store ground truth depth maps
alpha, R_c, R_cbt_bc, delta_t, gravities, velocities = [], [], [], [], [], []

avg_velocity = []

for idx, sample in enumerate(dataset_trajectory_2):
    # Standard processing
    for key, ipt in sample.items():
        if key in [("preint_imu", -1, 0), ("preint_imu", 0, 1)]:
            for pkey, pipt in sample[key].items():
                sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                if pkey not in ["v_norm", "delta_t"]:
                    pipt = pipt.unsqueeze(0)
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                else:
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
        else:
            ipt = ipt.unsqueeze(0)
            sample[key] = ipt.to(device, dtype=torch.float32)

    if idx == 0:
        init_sample = sample
        init_image = sample[('color', 0, 0)]
        disp_gt = sample["disp_gt"]
        depth_gt_init = (height // 2) / disp_gt.to(torch.float32)
        depth_gt_init = torch.clamp(depth_gt_init, min_depth, max_depth)
        depth_gt_init = depth_gt_init[:, 0].detach()

    frames = sample[('color', 0, 0)]
    disp_gt = sample["disp_gt"]
    disp_gt = (height // 2) / disp_gt.to(torch.float32)
    disp_gt = torch.clamp(disp_gt, min_depth, max_depth)
    depth_gt_sample = disp_gt[:, 0].detach()

    # Store ground truth depth maps
    depth_gt_list.append(depth_gt_sample.cpu())

    # Collect frames for visualization
    fig, (ax1, ax2) = plt.subplots(figsize=(10, 10), nrows=1, ncols=2)
    frame_data = frames
    im_2 = ax1.imshow(frame_data[0].permute(1, 2, 0).cpu().detach().numpy())
    im_3 = ax2.imshow(depth_gt_sample.permute(1, 2, 0).cpu().numpy(), cmap='viridis_r')
    list_frames_traj_2.append(im_2)
    list_gt_frames_traj_2.append(im_3)
    plt.close(fig)

    # Get IMU parameters
    next_imu = sample[('preint_imu', -1, 0)]
    alpha.append(next_imu["alpha"])
    R_c.append(next_imu["R_c"])
    R_cbt_bc.append(next_imu["R_cbt_bc"])
    delta_t.append(next_imu["delta_t"])
    avg_velocity.append(next_imu["v_norm"])

# Compute average velocity
vel_2 = torch.mean(torch.stack(avg_velocity), dim=0)
print(f"Average velocity: {vel_2}")

# Add code to create video of the original trajectory
image_frames_traj_2 = [im.get_array() for im in list_frames_traj_2]
gt_frames_traj_2 = [im.get_array() for im in list_gt_frames_traj_2]

# Create a figure and axis for the animation
fig, ax = plt.subplots()
# Initialize the image on the axis (using the first frame's shape)
im = ax.imshow(image_frames_traj_2[0], cmap='viridis')

# Update function for animation
def update(frame):
    im.set_data(frame)
    return [im]

# Create an animation
ani = animation.FuncAnimation(
    fig, update, frames=image_frames_traj_2, interval=100, blit=True
)

# Save the animation as a video file (e.g., MP4)
ani.save('Trajectories_anim/trajectory_2.mp4', writer='ffmpeg', fps=25)
plt.close(fig)

# Depth prediction using the same method as the second file
TRANS_SCALE_FACTOR = 5.4
pair_inputs = [init_sample[('color', 0, 0)], init_sample[('color', 1, 0)]]

# Generate velocity and gravity inputs
velo_inputs = [models["velo_encoder"](torch.cat(pair_inputs, 1))]
gravity_inputs = [models["gravity_encoder"](torch.cat(pair_inputs, 1))]

velocity = models["velo"](velo_inputs)
gravity = models["gravity"](gravity_inputs)

R_cbt_bc[0] = R_cbt_bc[0]

# Get depth prediction
with torch.no_grad():
    output = models["depth"](models["encoder"](init_image))
    disp_trajectory_2 = output

# Adjusted depth prediction
pred_disp, _ = disp_to_depth(disp_trajectory_2[("disp", 0)], min_depth, max_depth)
pred_disp = pred_disp.cpu()[:, 0]
pred_depth = 1 / pred_disp

# Apply median scaling
ratio = np.median(depth_gt_init.cpu().numpy()) / np.median(pred_depth.cpu().numpy())
pred_depth *= ratio
pred_depth = pred_depth.to(init_image.device).unsqueeze(0)

# Compute errors between predicted and ground truth depths
##########################################################
pred_depth_tmp = pred_depth.squeeze()
pred_depth_tmp = pred_depth_tmp.detach().cpu().numpy()
depth_gt_init_tmp = depth_gt_init.squeeze()
depth_gt_init_tmp = depth_gt_init_tmp.detach().cpu().numpy()

mask = (depth_gt_init_tmp > min_depth) & (depth_gt_init_tmp < max_depth)
pred_depth_flat = pred_depth_tmp[mask]

gt_depth_flat = depth_gt_init_tmp[mask]
print(f"init error : {compute_errors(gt_depth_flat, pred_depth_flat)[2]}")
##########################################################

# Generate poses and depth trajectory
poses_depth = get_poses_depth(dataset_trajectory_2, models, alpha, R_c, R_cbt_bc, delta_t, TRANS_SCALE_FACTOR)
depth_after_imu, depth_init_trajectory_2, images_after_imu = get_depth_trajectory(
    batch_size, init_image, pred_depth, poses_depth, init_sample
)

# Save depth video
get_video_depth(depth_after_imu, "Depth_anim/depth_trajectory_2.mp4")

# Function to compute errors
def compute_errors(gt, pred):
    """Compute error metrics between predicted and ground truth depths."""
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25     ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse = np.sqrt(((gt - pred) ** 2).mean())
    rmse_log = np.sqrt(((np.log(gt) - np.log(pred)) ** 2).mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)
    sq_rel = np.mean(((gt - pred) ** 2) / gt)

    return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3

# Now, compute errors between the predicted depth maps and ground truth
# Collect predicted depth maps from depth_after_imu
predicted_depth_maps = [depth.cpu().squeeze(0) for depth in depth_after_imu]

errors_trajectory_2 = []

for pred_depth_map, gt_depth_map in zip(predicted_depth_maps, depth_gt_list):
    pred_depth_map_np = pred_depth_map.detach().numpy() 
    gt_depth_map_np = gt_depth_map.numpy()
    
    # Apply mask to valid depth values
    mask = (gt_depth_map_np > min_depth) & (gt_depth_map_np < max_depth)
    
    pred_depth_flat = pred_depth_map_np[mask]
    gt_depth_flat = gt_depth_map_np[mask]
    
    # Ensure predicted depths are within valid range
    pred_depth_flat = np.clip(pred_depth_flat, min_depth, max_depth)
    
    # Compute error metrics
    errors_trajectory_2.append(compute_errors(gt_depth_flat, pred_depth_flat))

# Compute mean errors
mean_errors = np.mean(errors_trajectory_2, axis=0)

print("\n  " + ("{:>8} | " * 7).format("abs_rel", "sq_rel", "rmse", "rmse_log", "a1", "a2", "a3"))
print(("{: 8.3f}  " * 7).format(*mean_errors))


In [ ]:
rmse_trajectory_2 = [item[2] for item in errors_trajectory_2]
plt.title("RMSE as prediction horizon increases")
plt.plot(rmse_trajectory_2)
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")

In [ ]:
np.save('rmse_trajectories/rmse_trajectory_2.npy', rmse_trajectory_2)

In [ ]:
np.save("velocities/vel_2.npy", vel_2)

#### Trajectory 3

In [ ]:
# Main code
# INIT DEPTH PREDICTION
min_depth = 0.1
max_depth = 80

height = 512
width = 512
batch_size = 1

backproject_depth = BackprojectDepth(batch_size, height, width)
project_3d = Project3D(batch_size, height, width)
fraction_trajectory_3 = train_filenames[1833:1833+100]

dataset = datasets.midair.MidAirDataset
fpath = os.path.join("splits", "eigen_midair", "{}_files.txt")
data_path = "MidAir"

print("Creating dataset trajectory 1")
dataset_trajectory_3 = dataset(
    data_path, fraction_trajectory_3, 
    height, width,
    [0, -1, 1],
    select_file_names=True,
    num_scales=1, 
    use_imu=True,
    k_imu_clip=5, is_train=False, img_ext='.jpg' 
)   

In [ ]:
# Initialize lists to store data
dataloader_trajectory_3 = DataLoader(dataset_trajectory_3, 1, shuffle=False, num_workers=0, pin_memory=True, drop_last=False)
list_frames_traj_3 = []
list_gt_frames_traj_3 = []
depth_gt_list = []  # List to store ground truth depth maps
alpha, R_c, R_cbt_bc, delta_t, gravities, velocities = [], [], [], [], [], []

avg_velocity = []

for idx, sample in enumerate(dataset_trajectory_3):
    # Standard processing
    for key, ipt in sample.items():
        if key in [("preint_imu", -1, 0), ("preint_imu", 0, 1)]:
            for pkey, pipt in sample[key].items():
                sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                if pkey not in ["v_norm", "delta_t"]:
                    pipt = pipt.unsqueeze(0)
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                else:
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
        else:
            ipt = ipt.unsqueeze(0)
            sample[key] = ipt.to(device, dtype=torch.float32)

    if idx == 0:
        init_sample = sample
        init_image = sample[('color', 0, 0)]
        depth_gt = sample["disp_gt"]
        depth_gt = (height // 2) / depth_gt.to(torch.float32)
        depth_gt = torch.clamp(depth_gt, min_depth, max_depth)
        depth_gt = depth_gt[:, 0].detach()

    frames = sample[('color', 0, 0)]
    disp_gt = sample["disp_gt"]
    disp_gt = (height // 2) / disp_gt.to(torch.float32)
    disp_gt = torch.clamp(disp_gt, min_depth, max_depth)
    depth_gt_sample = disp_gt[:, 0].detach()

    # Store ground truth depth maps
    depth_gt_list.append(depth_gt_sample.cpu())

    # Collect frames for visualization
    fig, (ax1, ax2) = plt.subplots(figsize=(10, 10), nrows=1, ncols=2)
    frame_data = frames
    im_3 = ax1.imshow(frame_data[0].permute(1, 2, 0).cpu().detach().numpy())
    im_4 = ax2.imshow(depth_gt_sample.permute(1, 2, 0).cpu().numpy(), cmap='viridis_r')
    list_frames_traj_3.append(im_3)
    list_gt_frames_traj_3.append(im_4)
    plt.close(fig)

    # Get IMU parameters
    next_imu = sample[('preint_imu', -1, 0)]
    alpha.append(next_imu["alpha"])
    R_c.append(next_imu["R_c"])
    R_cbt_bc.append(next_imu["R_cbt_bc"])
    delta_t.append(next_imu["delta_t"])
    avg_velocity.append(next_imu["v_norm"])

# Compute average velocity
vel_3 = torch.mean(torch.stack(avg_velocity), dim=0)
print(f"Average velocity: {vel_3}")

# Add code to create video of the original trajectory
image_frames_traj_3 = [im.get_array() for im in list_frames_traj_3]
gt_frames_traj_3 = [im.get_array() for im in list_gt_frames_traj_3]

# Create a figure and axis for the animation
fig, ax = plt.subplots()
# Initialize the image on the axis (using the first frame's shape)
im = ax.imshow(image_frames_traj_3[0], cmap='viridis')

# Update function for animation
def update(frame):
    im.set_data(frame)
    return [im]

# Create an animation
ani = animation.FuncAnimation(
    fig, update, frames=image_frames_traj_3, interval=100, blit=True
)

# Save the animation as a video file (e.g., MP4)
ani.save('Trajectories_anim/trajectory_3.mp4', writer='ffmpeg', fps=25)
plt.close(fig)

# Depth prediction using the same method as the second file
TRANS_SCALE_FACTOR = 5.4
pair_inputs = [init_sample[('color', 0, 0)], init_sample[('color', 1, 0)]]

# Generate velocity and gravity inputs
velo_inputs = [models["velo_encoder"](torch.cat(pair_inputs, 1))]
gravity_inputs = [models["gravity_encoder"](torch.cat(pair_inputs, 1))]

velocity = models["velo"](velo_inputs)
gravity = models["gravity"](gravity_inputs)

R_cbt_bc[0] = R_cbt_bc[0]

# Get depth prediction
with torch.no_grad():
    output = models["depth"](models["encoder"](init_image))
    disp_trajectory_3 = output

# Adjusted depth prediction
pred_disp, _ = disp_to_depth(disp_trajectory_3[("disp", 0)], min_depth, max_depth)
pred_disp = pred_disp.cpu()[:, 0]
pred_depth = 1 / pred_disp

# Apply median scaling
ratio = np.median(depth_gt.cpu().numpy()) / np.median(pred_depth.cpu().numpy())
pred_depth *= ratio
pred_depth = pred_depth.to(init_image.device).unsqueeze(0)

# Ensure depth values are within valid range
pred_depth = torch.clamp(pred_depth, min_depth, max_depth)

# Generate poses and depth trajectory
poses_depth = get_poses_depth(dataset_trajectory_3, models, alpha, R_c, R_cbt_bc, delta_t, TRANS_SCALE_FACTOR)
depth_after_imu, depth_init_trajectory_3, images_after_imu = get_depth_trajectory(
    batch_size, init_image, disp_trajectory_3[("disp", 0)], poses_depth, init_sample
)

# Save depth video
get_video_depth(depth_after_imu, "Depth_anim/depth_trajectory_3.mp4")

# Function to compute errors
def compute_errors(gt, pred):
    """Compute error metrics between predicted and ground truth depths."""
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25     ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse = np.sqrt(((gt - pred) ** 2).mean())
    rmse_log = np.sqrt(((np.log(gt) - np.log(pred)) ** 2).mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)
    sq_rel = np.mean(((gt - pred) ** 2) / gt)

    return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3

# Now, compute errors between the predicted depth maps and ground truth
# Collect predicted depth maps from depth_after_imu
predicted_depth_maps = [depth.cpu().squeeze(0) for depth in depth_after_imu]

errors_trajectory_3 = []

for pred_depth_map, gt_depth_map in zip(predicted_depth_maps, depth_gt_list):
    pred_depth_map_np = pred_depth_map.detach().numpy() 
    gt_depth_map_np = gt_depth_map.numpy()
    
    # Apply mask to valid depth values
    mask = (gt_depth_map_np > min_depth) & (gt_depth_map_np < max_depth)
    
    pred_depth_flat = pred_depth_map_np[mask]
    gt_depth_flat = gt_depth_map_np[mask]
    
    # Ensure predicted depths are within valid range
    pred_depth_flat = np.clip(pred_depth_flat, min_depth, max_depth)
    
    # Compute error metrics
    errors_trajectory_3.append(compute_errors(gt_depth_flat, pred_depth_flat))

# Compute mean errors
mean_errors = np.mean(errors_trajectory_3, axis=0)

print("\n  " + ("{:>8} | " * 7).format("abs_rel", "sq_rel", "rmse", "rmse_log", "a1", "a2", "a3"))
print(("{: 8.3f}  " * 7).format(*mean_errors))


In [ ]:
rmse_trajectory_3 = [item[2] for item in errors_trajectory_3]
plt.title("RMSE as prediction horizon increases")
plt.plot(rmse_trajectory_3)
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")

In [ ]:
np.save('rmse_trajectories/rmse_trajectory_3.npy', rmse_trajectory_3)

In [ ]:
np.save("velocities/vel_3.npy", vel_3)

#### Trajectory 4

In [ ]:
# Main code
# INIT DEPTH PREDICTION
min_depth = 0.1
max_depth = 80

height = 512
width = 512
batch_size = 1

backproject_depth = BackprojectDepth(batch_size, height, width)
project_3d = Project3D(batch_size, height, width)
fraction_trajectory_4 = train_filenames[9349:9349+100]

dataset = datasets.midair.MidAirDataset
fpath = os.path.join("splits", "eigen_midair", "{}_files.txt")
data_path = "MidAir"

print("Creating dataset trajectory 1")
dataset_trajectory_4 = dataset(
    data_path, fraction_trajectory_4, 
    height, width,
    [0, -1, 1],
    select_file_names=True,
    num_scales=1, 
    use_imu=True,
    k_imu_clip=5, is_train=False, img_ext='.jpg' 
)   

In [ ]:
# Initialize lists to store data
dataloader_trajectory_4 = DataLoader(dataset_trajectory_4, 1, shuffle=False, num_workers=0, pin_memory=True, drop_last=False)
list_frames_traj_4 = []
list_gt_frames_traj_4 = []
depth_gt_list = []  # List to store ground truth depth maps
alpha, R_c, R_cbt_bc, delta_t, gravities, velocities = [], [], [], [], [], []

avg_velocity = []

for idx, sample in enumerate(dataset_trajectory_4):
    # Standard processing
    for key, ipt in sample.items():
        if key in [("preint_imu", -1, 0), ("preint_imu", 0, 1)]:
            for pkey, pipt in sample[key].items():
                sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                if pkey not in ["v_norm", "delta_t"]:
                    pipt = pipt.unsqueeze(0)
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                else:
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
        else:
            ipt = ipt.unsqueeze(0)
            sample[key] = ipt.to(device, dtype=torch.float32)

    if idx == 0:
        init_sample = sample
        init_image = sample[('color', 0, 0)]
        depth_gt = sample["disp_gt"]
        depth_gt = (height // 2) / depth_gt.to(torch.float32)
        depth_gt = torch.clamp(depth_gt, min_depth, max_depth)
        depth_gt = depth_gt[:, 0].detach()

    frames = sample[('color', 0, 0)]
    disp_gt = sample["disp_gt"]
    disp_gt = (height // 2) / disp_gt.to(torch.float32)
    disp_gt = torch.clamp(disp_gt, min_depth, max_depth)
    depth_gt_sample = disp_gt[:, 0].detach()

    # Store ground truth depth maps
    depth_gt_list.append(depth_gt_sample.cpu())

    # Collect frames for visualization
    fig, (ax1, ax2) = plt.subplots(figsize=(10, 10), nrows=1, ncols=2)
    frame_data = frames
    im_4 = ax1.imshow(frame_data[0].permute(1, 2, 0).cpu().detach().numpy())
    im_5 = ax2.imshow(depth_gt_sample.permute(1, 2, 0).cpu().numpy(), cmap='viridis_r')
    list_frames_traj_4.append(im_4)
    list_gt_frames_traj_4.append(im_5)
    plt.close(fig)

    # Get IMU parameters
    next_imu = sample[('preint_imu', -1, 0)]
    alpha.append(next_imu["alpha"])
    R_c.append(next_imu["R_c"])
    R_cbt_bc.append(next_imu["R_cbt_bc"])
    delta_t.append(next_imu["delta_t"])
    avg_velocity.append(next_imu["v_norm"])

# Compute average velocity
vel_4 = torch.mean(torch.stack(avg_velocity), dim=0)
print(f"Average velocity: {vel_4}")

# Add code to create video of the original trajectory
image_frames_traj_4 = [im.get_array() for im in list_frames_traj_4]
gt_frames_traj_4 = [im.get_array() for im in list_gt_frames_traj_4]

# Create a figure and axis for the animation
fig, ax = plt.subplots()
# Initialize the image on the axis (using the first frame's shape)
im = ax.imshow(image_frames_traj_4[0], cmap='viridis')

# Update function for animation
def update(frame):
    im.set_data(frame)
    return [im]

# Create an animation
ani = animation.FuncAnimation(
    fig, update, frames=image_frames_traj_4, interval=100, blit=True
)

# Save the animation as a video file (e.g., MP4)
ani.save('Trajectories_anim/trajectory_4.mp4', writer='ffmpeg', fps=25)
plt.close(fig)

# Depth prediction using the same method as the second file
TRANS_SCALE_FACTOR = 5.4
pair_inputs = [init_sample[('color', 0, 0)], init_sample[('color', 1, 0)]]

# Generate velocity and gravity inputs
velo_inputs = [models["velo_encoder"](torch.cat(pair_inputs, 1))]
gravity_inputs = [models["gravity_encoder"](torch.cat(pair_inputs, 1))]

velocity = models["velo"](velo_inputs)
gravity = models["gravity"](gravity_inputs)

R_cbt_bc[0] = R_cbt_bc[0]

# Get depth prediction
with torch.no_grad():
    output = models["depth"](models["encoder"](init_image))
    disp_trajectory_4 = output

# Adjusted depth prediction
pred_disp, _ = disp_to_depth(disp_trajectory_4[("disp", 0)], min_depth, max_depth)
pred_disp = pred_disp.cpu()[:, 0]
pred_depth = 1 / pred_disp

# Apply median scaling
ratio = np.median(depth_gt.cpu().numpy()) / np.median(pred_depth.cpu().numpy())
pred_depth *= ratio
pred_depth = pred_depth.to(init_image.device).unsqueeze(0)

# Ensure depth values are within valid range
pred_depth = torch.clamp(pred_depth, min_depth, max_depth)

# Generate poses and depth trajectory
poses_depth = get_poses_depth(dataset_trajectory_4, models, alpha, R_c, R_cbt_bc, delta_t, TRANS_SCALE_FACTOR)
depth_after_imu, depth_init_trajectory_4, images_after_imu = get_depth_trajectory(
    batch_size, init_image, disp_trajectory_4[("disp", 0)], poses_depth, init_sample
)

# Save depth video
get_video_depth(depth_after_imu, "Depth_anim/depth_trajectory_4.mp4")

# Function to compute errors
def compute_errors(gt, pred):
    """Compute error metrics between predicted and ground truth depths."""
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25     ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse_trajectory_4 = np.sqrt(((gt - pred) ** 2).mean())
    rmse_log = np.sqrt(((np.log(gt) - np.log(pred)) ** 2).mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)
    sq_rel = np.mean(((gt - pred) ** 2) / gt)

    return abs_rel, sq_rel, rmse_trajectory_4, rmse_log, a1, a2, a3

# Now, compute errors between the predicted depth maps and ground truth
# Collect predicted depth maps from depth_after_imu
predicted_depth_maps = [depth.cpu().squeeze(0) for depth in depth_after_imu]

errors_trajectory_4 = []

for pred_depth_map, gt_depth_map in zip(predicted_depth_maps, depth_gt_list):
    pred_depth_map_np = pred_depth_map.detach().numpy() 
    gt_depth_map_np = gt_depth_map.numpy()
    
    # Apply mask to valid depth values
    mask = (gt_depth_map_np > min_depth) & (gt_depth_map_np < max_depth)
    
    pred_depth_flat = pred_depth_map_np[mask]
    gt_depth_flat = gt_depth_map_np[mask]
    
    # Ensure predicted depths are within valid range
    pred_depth_flat = np.clip(pred_depth_flat, min_depth, max_depth)
    
    # Compute error metrics
    errors_trajectory_4.append(compute_errors(gt_depth_flat, pred_depth_flat))

# Compute mean errors
mean_errors = np.mean(errors_trajectory_4, axis=0)

print("\n  " + ("{:>8} | " * 7).format("abs_rel", "sq_rel", "rmse_trajectory_4", "rmse_log", "a1", "a2", "a3"))
print(("{: 8.3f}  " * 7).format(*mean_errors))


In [ ]:
rmse_trajectory_4 = [item[2] for item in errors_trajectory_4]
plt.title("RMSE as prediction horizon increases")
plt.plot(rmse_trajectory_4)
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")

#### Trajectory 5

In [ ]:
# Main code
# INIT DEPTH PREDICTION
min_depth = 0.1
max_depth = 80

height = 512
width = 512
batch_size = 1

backproject_depth = BackprojectDepth(batch_size, height, width)
project_3d = Project3D(batch_size, height, width)
fraction_trajectory_5 = train_filenames[9632:9632+100]

dataset = datasets.midair.MidAirDataset
fpath = os.path.join("splits", "eigen_midair", "{}_files.txt")
data_path = "MidAir"

print("Creating dataset trajectory 1")
dataset_trajectory_5 = dataset(
    data_path, fraction_trajectory_5, 
    height, width,
    [0, -1, 1],
    select_file_names=True,
    num_scales=1, 
    use_imu=True,
    k_imu_clip=5, is_train=False, img_ext='.jpg' 
)   

In [ ]:
# Initialize lists to store data
dataloader_trajectory_5 = DataLoader(dataset_trajectory_5, 1, shuffle=False, num_workers=0, pin_memory=True, drop_last=False)
list_frames_traj_5 = []
list_gt_frames_traj_5 = []
depth_gt_list = []  # List to store ground truth depth maps
alpha, R_c, R_cbt_bc, delta_t, gravities, velocities = [], [], [], [], [], []

avg_velocity = []

for idx, sample in enumerate(dataset_trajectory_5):
    # Standard processing
    for key, ipt in sample.items():
        if key in [("preint_imu", -1, 0), ("preint_imu", 0, 1)]:
            for pkey, pipt in sample[key].items():
                sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                if pkey not in ["v_norm", "delta_t"]:
                    pipt = pipt.unsqueeze(0)
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                else:
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
        else:
            ipt = ipt.unsqueeze(0)
            sample[key] = ipt.to(device, dtype=torch.float32)

    if idx == 0:
        init_sample = sample
        init_image = sample[('color', 0, 0)]
        depth_gt = sample["disp_gt"]
        depth_gt = (height // 2) / depth_gt.to(torch.float32)
        depth_gt = torch.clamp(depth_gt, min_depth, max_depth)
        depth_gt = depth_gt[:, 0].detach()

    frames = sample[('color', 0, 0)]
    disp_gt = sample["disp_gt"]
    disp_gt = (height // 2) / disp_gt.to(torch.float32)
    disp_gt = torch.clamp(disp_gt, min_depth, max_depth)
    depth_gt_sample = disp_gt[:, 0].detach()

    # Store ground truth depth maps
    depth_gt_list.append(depth_gt_sample.cpu())

    # Collect frames for visualization
    fig, (ax1, ax2) = plt.subplots(figsize=(10, 10), nrows=1, ncols=2)
    frame_data = frames
    im_5 = ax1.imshow(frame_data[0].permute(1, 2, 0).cpu().detach().numpy())
    im_6 = ax2.imshow(depth_gt_sample.permute(1, 2, 0).cpu().numpy(), cmap='viridis_r')
    list_frames_traj_5.append(im_5)
    list_gt_frames_traj_5.append(im_6)
    plt.close(fig)

    # Get IMU parameters
    next_imu = sample[('preint_imu', -1, 0)]
    alpha.append(next_imu["alpha"])
    R_c.append(next_imu["R_c"])
    R_cbt_bc.append(next_imu["R_cbt_bc"])
    delta_t.append(next_imu["delta_t"])
    avg_velocity.append(next_imu["v_norm"])

# Compute average velocity
vel_5 = torch.mean(torch.stack(avg_velocity), dim=0)
print(f"Average velocity: {vel_5}")

# Add code to create video of the original trajectory
image_frames_traj_5 = [im.get_array() for im in list_frames_traj_5]
gt_frames_traj_5 = [im.get_array() for im in list_gt_frames_traj_5]

# Create a figure and axis for the animation
fig, ax = plt.subplots()
# Initialize the image on the axis (using the first frame's shape)
im = ax.imshow(image_frames_traj_5[0], cmap='viridis')

# Update function for animation
def update(frame):
    im.set_data(frame)
    return [im]

# Create an animation
ani = animation.FuncAnimation(
    fig, update, frames=image_frames_traj_5, interval=100, blit=True
)

# Save the animation as a video file (e.g., MP4)
ani.save('Trajectories_anim/trajectory_5.mp4', writer='ffmpeg', fps=25)
plt.close(fig)

# Depth prediction using the same method as the second file
TRANS_SCALE_FACTOR = 5.4
pair_inputs = [init_sample[('color', 0, 0)], init_sample[('color', 1, 0)]]

# Generate velocity and gravity inputs
velo_inputs = [models["velo_encoder"](torch.cat(pair_inputs, 1))]
gravity_inputs = [models["gravity_encoder"](torch.cat(pair_inputs, 1))]

velocity = models["velo"](velo_inputs)
gravity = models["gravity"](gravity_inputs)

R_cbt_bc[0] = R_cbt_bc[0]

# Get depth prediction
with torch.no_grad():
    output = models["depth"](models["encoder"](init_image))
    disp_trajectory_5 = output

# Adjusted depth prediction
pred_disp, _ = disp_to_depth(disp_trajectory_5[("disp", 0)], min_depth, max_depth)
pred_disp = pred_disp.cpu()[:, 0]
pred_depth = 1 / pred_disp

# Apply median scaling
ratio = np.median(depth_gt.cpu().numpy()) / np.median(pred_depth.cpu().numpy())
pred_depth *= ratio
pred_depth = pred_depth.to(init_image.device).unsqueeze(0)

# Ensure depth values are within valid range
pred_depth = torch.clamp(pred_depth, min_depth, max_depth)

# Generate poses and depth trajectory
poses_depth = get_poses_depth(dataset_trajectory_5, models, alpha, R_c, R_cbt_bc, delta_t, TRANS_SCALE_FACTOR)
depth_after_imu, depth_init_trajectory_5, images_after_imu = get_depth_trajectory(
    batch_size, init_image, disp_trajectory_5[("disp", 0)], poses_depth, init_sample
)

# Save depth video
get_video_depth(depth_after_imu, "Depth_anim/depth_trajectory_5.mp4")

# Function to compute errors
def compute_errors(gt, pred):
    """Compute error metrics between predicted and ground truth depths."""
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25     ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse_trajectory_5 = np.sqrt(((gt - pred) ** 2).mean())
    rmse_log = np.sqrt(((np.log(gt) - np.log(pred)) ** 2).mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)
    sq_rel = np.mean(((gt - pred) ** 2) / gt)

    return abs_rel, sq_rel, rmse_trajectory_5, rmse_log, a1, a2, a3

# Now, compute errors between the predicted depth maps and ground truth
# Collect predicted depth maps from depth_after_imu
predicted_depth_maps = [depth.cpu().squeeze(0) for depth in depth_after_imu]

errors_trajectory_5 = []

for pred_depth_map, gt_depth_map in zip(predicted_depth_maps, depth_gt_list):
    pred_depth_map_np = pred_depth_map.detach().numpy() 
    gt_depth_map_np = gt_depth_map.numpy()
    
    # Apply mask to valid depth values
    mask = (gt_depth_map_np > min_depth) & (gt_depth_map_np < max_depth)
    
    pred_depth_flat = pred_depth_map_np[mask]
    gt_depth_flat = gt_depth_map_np[mask]
    
    # Ensure predicted depths are within valid range
    pred_depth_flat = np.clip(pred_depth_flat, min_depth, max_depth)
    
    # Compute error metrics
    errors_trajectory_5.append(compute_errors(gt_depth_flat, pred_depth_flat))

# Compute mean errors
mean_errors = np.mean(errors_trajectory_5, axis=0)

print("\n  " + ("{:>8} | " * 7).format("abs_rel", "sq_rel", "rmse_trajectory_5", "rmse_log", "a1", "a2", "a3"))
print(("{: 8.3f}  " * 7).format(*mean_errors))


In [ ]:
rmse_trajectory_5 = [item[2] for item in errors_trajectory_5]
plt.title("RMSE as prediction horizon increases")
plt.plot(rmse_trajectory_5)
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")

#### Trajectory 6

In [ ]:
dataset = datasets.midair.MidAirDataset

fpath = os.path.join("splits","eigen_midair", "{}_files.txt")
train_filenames = readlines(fpath.format("test"))

device = torch.device("cpu")

In [ ]:
# Main code
# INIT DEPTH PREDICTION
min_depth = 0.1
max_depth = 80

height = 512
width = 512
batch_size = 1

backproject_depth = BackprojectDepth(batch_size, height, width)
project_3d = Project3D(batch_size, height, width)
fraction_trajectory_6 = train_filenames[9684:9684+100]

dataset = datasets.midair.MidAirDataset
fpath = os.path.join("splits", "eigen_midair", "{}_files.txt")
data_path = "MidAir"

print("Creating dataset trajectory 1")
dataset_trajectory_6 = dataset(
    data_path, fraction_trajectory_6, 
    height, width,
    [0, -1, 1],
    select_file_names=True,
    num_scales=1, 
    use_imu=True,
    k_imu_clip=5, is_train=False, img_ext='.jpg' 
)   

In [ ]:
# Initialize lists to store data
dataloader_trajectory_6 = DataLoader(dataset_trajectory_6, 1, shuffle=False, num_workers=0, pin_memory=True, drop_last=False)
list_frames_traj_6 = []
list_gt_frames_traj_6 = []
depth_gt_list = []  # List to store ground truth depth maps
alpha, R_c, R_cbt_bc, delta_t, gravities, velocities = [], [], [], [], [], []

avg_velocity = []

for idx, sample in enumerate(dataset_trajectory_6):
    # Standard processing
    for key, ipt in sample.items():
        if key in [("preint_imu", -1, 0), ("preint_imu", 0, 1)]:
            for pkey, pipt in sample[key].items():
                sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                if pkey not in ["v_norm", "delta_t"]:
                    pipt = pipt.unsqueeze(0)
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                else:
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
        else:
            ipt = ipt.unsqueeze(0)
            sample[key] = ipt.to(device, dtype=torch.float32)

    if idx == 0:
        init_sample = sample
        init_image = sample[('color', 0, 0)]
        disp_gt_batch = sample["disp_gt"]
        depth_gt_init = (height // 2) / disp_gt_batch.to(torch.float32)
        depth_gt_init = torch.clamp(depth_gt_init, min_depth, max_depth)
        depth_gt_init = depth_gt_init[:, 0].detach()

    frames = sample[('color', 0, 0)]
    disp_gt = sample["disp_gt"]
    disp_gt = (height // 2) / disp_gt.to(torch.float32)
    disp_gt = torch.clamp(disp_gt, min_depth, max_depth)
    depth_gt_sample = disp_gt[:, 0].detach()

    # Store ground truth depth maps
    depth_gt_list.append(depth_gt_sample.cpu())

    # Collect frames for visualization
    fig, (ax1, ax2) = plt.subplots(figsize=(10, 10), nrows=1, ncols=2)
    frame_data = frames
    im_6 = ax1.imshow(frame_data[0].permute(1, 2, 0).cpu().detach().numpy())
    im_7 = ax2.imshow(depth_gt_sample.permute(1, 2, 0).cpu().numpy(), cmap='viridis_r')
    list_frames_traj_6.append(im_6)
    list_gt_frames_traj_6.append(im_7)
    plt.close(fig)

    # Get IMU parameters
    next_imu = sample[('preint_imu', -1, 0)]
    alpha.append(next_imu["alpha"])
    R_c.append(next_imu["R_c"])
    R_cbt_bc.append(next_imu["R_cbt_bc"])
    delta_t.append(next_imu["delta_t"])
    avg_velocity.append(next_imu["v_norm"])

# Compute average velocity
vel_6 = torch.mean(torch.stack(avg_velocity), dim=0)
print(f"Average velocity: {vel_6}")

# Add code to create video of the original trajectory
image_frames_traj_6 = [im.get_array() for im in list_frames_traj_6]
gt_frames_traj_6 = [im.get_array() for im in list_gt_frames_traj_6]

# Create a figure and axis for the animation
fig, ax = plt.subplots()
# Initialize the image on the axis (using the first frame's shape)
im = ax.imshow(image_frames_traj_6[0], cmap='viridis')

# Update function for animation
def update(frame):
    im.set_data(frame)
    return [im]

# Create an animation
ani = animation.FuncAnimation(
    fig, update, frames=image_frames_traj_6, interval=100, blit=True
)

# Save the animation as a video file (e.g., MP4)
ani.save('Trajectories_anim/trajectory_6.mp4', writer='ffmpeg', fps=25)
plt.close(fig)

# Depth prediction using the same method as the second file
TRANS_SCALE_FACTOR = 5.4
pair_inputs = [init_sample[('color', 0, 0)], init_sample[('color', 1, 0)]]

# Generate velocity and gravity inputs
velo_inputs = [models["velo_encoder"](torch.cat(pair_inputs, 1))]
gravity_inputs = [models["gravity_encoder"](torch.cat(pair_inputs, 1))]

velocity = models["velo"](velo_inputs)
gravity = models["gravity"](gravity_inputs)

R_cbt_bc[0] = R_cbt_bc[0]

# Get depth prediction
with torch.no_grad():
    output = models["depth"](models["encoder"](init_image))
    disp_trajectory_6 = output

# Adjusted depth prediction
pred_disp, _ = disp_to_depth(disp_trajectory_6[("disp", 0)], min_depth, max_depth)
pred_disp = pred_disp.cpu()[:, 0]
pred_depth = 1 / pred_disp

# Apply median scaling
ratio = np.median(depth_gt_init.cpu().numpy()) / np.median(pred_depth.cpu().numpy())
pred_depth *= ratio
pred_depth = pred_depth.to(init_image.device).unsqueeze(0)

# Compute errors between predicted and ground truth depths
##########################################################
pred_depth_tmp = pred_depth.squeeze()
pred_depth_tmp = pred_depth_tmp.detach().cpu().numpy()
depth_gt_init_tmp = depth_gt_init.squeeze()
depth_gt_init_tmp = depth_gt_init_tmp.detach().cpu().numpy()

mask = (depth_gt_init_tmp > min_depth) & (depth_gt_init_tmp < max_depth)
pred_depth_flat = pred_depth_tmp[mask]

gt_depth_flat = depth_gt_init_tmp[mask]
print(f"init error : {compute_errors(gt_depth_flat, pred_depth_flat)[2]}")
##########################################################


# Generate poses and depth trajectory
poses_depth = get_poses_depth(dataset_trajectory_6, models, alpha, R_c, R_cbt_bc, delta_t, TRANS_SCALE_FACTOR)
depth_after_imu, depth_init_trajectory_6, images_after_imu = get_depth_trajectory(
    batch_size, init_image, pred_depth, poses_depth, init_sample
)

# Save depth video
get_video_depth(depth_after_imu, "Depth_anim/depth_trajectory_6_2.mp4")

# Function to compute errors
def compute_errors(gt, pred):
    """Compute error metrics between predicted and ground truth depths."""
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25     ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse_trajectory_6 = np.sqrt(((gt - pred) ** 2).mean())
    rmse_log = np.sqrt(((np.log(gt) - np.log(pred)) ** 2).mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)
    sq_rel = np.mean(((gt - pred) ** 2) / gt)

    return abs_rel, sq_rel, rmse_trajectory_6, rmse_log, a1, a2, a3

# Now, compute errors between the predicted depth maps and ground truth
# Collect predicted depth maps from depth_after_imu
predicted_depth_maps = [depth.cpu().squeeze(0) for depth in depth_after_imu]

errors_trajectory_6 = []

for pred_depth_map, gt_depth_map in zip(predicted_depth_maps, depth_gt_list):
    pred_depth_map_np = pred_depth_map.detach().numpy() 
    gt_depth_map_np = gt_depth_map.numpy()
    
    # Apply mask to valid depth values
    mask = (gt_depth_map_np > min_depth) & (gt_depth_map_np < max_depth)
    
    pred_depth_flat = pred_depth_map_np[mask]
    gt_depth_flat = gt_depth_map_np[mask]
    
    # Ensure predicted depths are within valid range
    pred_depth_flat = np.clip(pred_depth_flat, min_depth, max_depth)
    
    # Compute error metrics
    errors_trajectory_6.append(compute_errors(gt_depth_flat, pred_depth_flat))

# Compute mean errors
mean_errors = np.mean(errors_trajectory_6, axis=0)

print("\n  " + ("{:>8} | " * 7).format("abs_rel", "sq_rel", "rmse_trajectory_6", "rmse_log", "a1", "a2", "a3"))
print(("{: 8.3f}  " * 7).format(*mean_errors))


In [ ]:
rmse_trajectory_6 = [item[2] for item in errors_trajectory_6]
plt.title("RMSE as prediction horizon increases")
plt.plot(rmse_trajectory_6)
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")

In [ ]:
np.save('rmse_trajectories/rmse_trajectory_6.npy', rmse_trajectory_6)

In [ ]:
np.save("velocities/vel_6.npy", vel_6)

In [ ]:
file_name = "000000.png"
idx2 = [0,5,10,15,20,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]
for i in range(len(image_frames_traj_6)):
    if i in idx2:
        image = image_frames_traj_6[i]
        save_file = file_name.split(".")[0] + f"_{i}.png"
        plt.imshow(image)
        plt.imsave(f"/Users/ibrahimhassan/Documents/Documents/thesis_DynaDepth/Report_images_1/{save_file}",image)
        

In [ ]:
file_name = "100000.png"
for i in range(len(depth_after_imu)):
    if i in idx2:
        image = depth_after_imu[i][0]
        save_file = file_name.split(".")[0] + f"_{i}.png"
        print(image.permute(1,2,0).detach().numpy().shape)
        plt.imshow(image.permute(1,2,0).detach().numpy())
        plt.imsave(f"/Users/ibrahimhassan/Documents/Documents/thesis_DynaDepth/Report_images_1/{save_file}",image.squeeze().detach().numpy(), cmap='viridis')
        

#### Trajectory 7

In [ ]:
print(os.path.join("splits", "eigen_midair", "{}_files.txt"))
fpath = os.path.join("splits","eigen_midair", "{}_files.txt")

train_filenames = readlines(fpath.format("train"))

train_filenames = sorted(train_filenames, key=custom_sort_key)

In [ ]:
# Main code
# INIT DEPTH PREDICTION
min_depth = 0.1
max_depth = 80

height = 512
width = 512
batch_size = 1

backproject_depth = BackprojectDepth(batch_size, height, width)
project_3d = Project3D(batch_size, height, width)
fraction_trajectory_7 = train_filenames[25588:25588+100]

dataset = datasets.midair.MidAirDataset
fpath = os.path.join("splits", "eigen_midair", "{}_files.txt")
data_path = "MidAir"

print("Creating dataset trajectory 1")
dataset_trajectory_7 = dataset(
    data_path, fraction_trajectory_7, 
    height, width,
    [0, -1, 1],
    select_file_names=True,
    num_scales=1, 
    use_imu=True,
    k_imu_clip=5, is_train=False, img_ext='.jpg' 
)   

In [ ]:
# Initialize lists to store data
dataloader_trajectory_7 = DataLoader(dataset_trajectory_7, 1, shuffle=False, num_workers=0, pin_memory=True, drop_last=False)
list_frames_traj_7 = []
list_gt_frames_traj_7 = []
depth_gt_list = []  # List to store ground truth depth maps
alpha, R_c, R_cbt_bc, delta_t, gravities, velocities = [], [], [], [], [], []

avg_velocity = []

for idx, sample in enumerate(dataset_trajectory_7):
    # Standard processing
    for key, ipt in sample.items():
        if key in [("preint_imu", -1, 0), ("preint_imu", 0, 1)]:
            for pkey, pipt in sample[key].items():
                sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                if pkey not in ["v_norm", "delta_t"]:
                    pipt = pipt.unsqueeze(0)
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                else:
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
        else:
            ipt = ipt.unsqueeze(0)
            sample[key] = ipt.to(device, dtype=torch.float32)

    if idx == 0:
        init_sample = sample
        init_image = sample[('color', 0, 0)]
        disp_gt_batch = sample["disp_gt"]
        depth_gt_init = (height // 2) / disp_gt_batch.to(torch.float32)
        depth_gt_init = torch.clamp(depth_gt_init, min_depth, max_depth)
        depth_gt_init = depth_gt_init[:, 0].detach()

    frames = sample[('color', 0, 0)]
    disp_gt = sample["disp_gt"]
    disp_gt = (height // 2) / disp_gt.to(torch.float32)
    disp_gt = torch.clamp(disp_gt, min_depth, max_depth)
    depth_gt_sample = disp_gt[:, 0].detach()

    # Store ground truth depth maps
    depth_gt_list.append(depth_gt_sample.cpu())

    # Collect frames for visualization
    fig, (ax1, ax2) = plt.subplots(figsize=(10, 10), nrows=1, ncols=2)
    frame_data = frames
    im_7 = ax1.imshow(frame_data[0].permute(1, 2, 0).cpu().detach().numpy())
    im_8 = ax2.imshow(depth_gt_sample.permute(1, 2, 0).cpu().numpy(), cmap='viridis_r')
    list_frames_traj_7.append(im_7)
    list_gt_frames_traj_7.append(im_8)
    plt.close(fig)

    # Get IMU parameters
    next_imu = sample[('preint_imu', -1, 0)]
    alpha.append(next_imu["alpha"])
    R_c.append(next_imu["R_c"])
    R_cbt_bc.append(next_imu["R_cbt_bc"])
    delta_t.append(next_imu["delta_t"])
    avg_velocity.append(next_imu["v_norm"])

# Compute average velocity
vel_7 = torch.mean(torch.stack(avg_velocity), dim=0)
print(f"Average velocity: {vel_7}")

# Add code to create video of the original trajectory
image_frames_traj_7 = [im.get_array() for im in list_frames_traj_7]
gt_frames_traj_7 = [im.get_array() for im in list_gt_frames_traj_7]

# Create a figure and axis for the animation
fig, ax = plt.subplots()
# Initialize the image on the axis (using the first frame's shape)
im = ax.imshow(image_frames_traj_7[0], cmap='viridis')

# Update function for animation
def update(frame):
    im.set_data(frame)
    return [im]

# Create an animation
ani = animation.FuncAnimation(
    fig, update, frames=image_frames_traj_7, interval=100, blit=True
)

# Save the animation as a video file (e.g., MP4)
ani.save('Trajectories_anim/trajectory_7.mp4', writer='ffmpeg', fps=25)
plt.close(fig)

# Depth prediction using the same method as the second file
TRANS_SCALE_FACTOR = 5.4
pair_inputs = [init_sample[('color', 0, 0)], init_sample[('color', 1, 0)]]

# Generate velocity and gravity inputs
velo_inputs = [models["velo_encoder"](torch.cat(pair_inputs, 1))]
gravity_inputs = [models["gravity_encoder"](torch.cat(pair_inputs, 1))]

velocity = models["velo"](velo_inputs)
gravity = models["gravity"](gravity_inputs)

R_cbt_bc[0] = R_cbt_bc[0]

# Get depth prediction
with torch.no_grad():
    output = models["depth"](models["encoder"](init_image))
    disp_trajectory_7 = output

# Adjusted depth prediction
pred_disp, _ = disp_to_depth(disp_trajectory_7[("disp", 0)], min_depth, max_depth)
pred_disp = pred_disp.cpu()[:, 0]
pred_depth = 1 / pred_disp

# Apply median scaling
ratio = np.median(depth_gt_init.cpu().numpy()) / np.median(pred_depth.cpu().numpy())
pred_depth *= ratio
pred_depth = pred_depth.to(init_image.device).unsqueeze(0)


# Compute errors between predicted and ground truth depths
##########################################################
pred_depth_tmp = pred_depth.squeeze()
pred_depth_tmp = pred_depth_tmp.detach().cpu().numpy()
depth_gt_init_tmp = depth_gt_init.squeeze()
depth_gt_init_tmp = depth_gt_init_tmp.detach().cpu().numpy()

mask = (depth_gt_init_tmp > min_depth) & (depth_gt_init_tmp < max_depth)
pred_depth_flat = pred_depth_tmp[mask]

gt_depth_flat = depth_gt_init_tmp[mask]
print(f"init error : {compute_errors(gt_depth_flat, pred_depth_flat)[2]}")
##########################################################

# Generate poses and depth trajectory
poses_depth = get_poses_depth(dataset_trajectory_7, models, alpha, R_c, R_cbt_bc, delta_t, TRANS_SCALE_FACTOR)
depth_after_imu, depth_init_trajectory_7, images_after_imu = get_depth_trajectory(
    batch_size, init_image,pred_depth, poses_depth, init_sample
)

# Save depth video
get_video_depth(depth_after_imu, "Depth_anim/depth_trajectory_7.mp4")

# Function to compute errors
def compute_errors(gt, pred):
    """Compute error metrics between predicted and ground truth depths."""
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25     ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse_trajectory_7 = np.sqrt(((gt - pred) ** 2).mean())
    rmse_log = np.sqrt(((np.log(gt) - np.log(pred)) ** 2).mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)
    sq_rel = np.mean(((gt - pred) ** 2) / gt)

    return abs_rel, sq_rel, rmse_trajectory_7, rmse_log, a1, a2, a3

# Now, compute errors between the predicted depth maps and ground truth
# Collect predicted depth maps from depth_after_imu
predicted_depth_maps = [depth.cpu().squeeze(0) for depth in depth_after_imu]

errors_trajectory_7 = []

for pred_depth_map, gt_depth_map in zip(predicted_depth_maps, depth_gt_list):
    pred_depth_map_np = pred_depth_map.detach().numpy() 
    gt_depth_map_np = gt_depth_map.numpy()
    
    # Apply mask to valid depth values
    mask = (gt_depth_map_np > min_depth) & (gt_depth_map_np < max_depth)
    
    pred_depth_flat = pred_depth_map_np[mask]
    gt_depth_flat = gt_depth_map_np[mask]
    
    # Ensure predicted depths are within valid range
    pred_depth_flat = np.clip(pred_depth_flat, min_depth, max_depth)
    
    # Compute error metrics
    errors_trajectory_7.append(compute_errors(gt_depth_flat, pred_depth_flat))

# Compute mean errors
mean_errors = np.mean(errors_trajectory_7, axis=0)

print("\n  " + ("{:>8} | " * 7).format("abs_rel", "sq_rel", "rmse_trajectory_7", "rmse_log", "a1", "a2", "a3"))
print(("{: 8.3f}  " * 7).format(*mean_errors))


In [ ]:
rmse_trajectory_7 = [item[2] for item in errors_trajectory_7]
plt.title("RMSE as prediction horizon increases")
plt.plot(rmse_trajectory_7)
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")

In [ ]:
rmse_trajectory_7[20]

In [ ]:
np.save('rmse_trajectories/rmse_trajectory_7.npy', rmse_trajectory_7)

In [ ]:
np.save("velocities/vel_7.npy", vel_7)

#### Trajectory 8

In [ ]:
print(os.path.join("splits", "eigen_midair", "{}_files.txt"))
fpath = os.path.join("splits","eigen_midair", "{}_files.txt")

train_filenames = readlines(fpath.format("train"))

train_filenames = sorted(train_filenames, key=custom_sort_key)

In [ ]:
# Main code
# INIT DEPTH PREDICTION
min_depth = 0.1
max_depth = 80

height = 512
width = 512
batch_size = 1

backproject_depth = BackprojectDepth(batch_size, height, width)
project_3d = Project3D(batch_size, height, width)
fraction_trajectory_8 = train_filenames[24106:24106+100]

dataset = datasets.midair.MidAirDataset
fpath = os.path.join("splits", "eigen_midair", "{}_files.txt")
data_path = "MidAir"

print("Creating dataset trajectory 1")
dataset_trajectory_8 = dataset(
    data_path, fraction_trajectory_8, 
    height, width,
    [0, -1, 1],
    select_file_names=True,
    num_scales=1, 
    use_imu=True,
    k_imu_clip=5, is_train=False, img_ext='.jpg' 
)   

In [ ]:
# Initialize lists to store data
dataloader_trajectory_8 = DataLoader(dataset_trajectory_8, 1, shuffle=False, num_workers=0, pin_memory=True, drop_last=False)
list_frames_traj_8 = []
list_gt_frames_traj_8 = []
depth_gt_list = []  # List to store ground truth depth maps
alpha, R_c, R_cbt_bc, delta_t, gravities, velocities = [], [], [], [], [], []

avg_velocity = []

for idx, sample in enumerate(dataset_trajectory_8):
    # Standard processing
    for key, ipt in sample.items():
        if key in [("preint_imu", -1, 0), ("preint_imu", 0, 1)]:
            for pkey, pipt in sample[key].items():
                sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                if pkey not in ["v_norm", "delta_t"]:
                    pipt = pipt.unsqueeze(0)
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                else:
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
        else:
            ipt = ipt.unsqueeze(0)
            sample[key] = ipt.to(device, dtype=torch.float32)

    if idx == 0:
        init_sample = sample
        init_image = sample[('color', 0, 0)]
        depth_gt = sample["disp_gt"]
        depth_gt = (height // 2) / depth_gt.to(torch.float32)
        depth_gt = torch.clamp(depth_gt, min_depth, max_depth)
        depth_gt = depth_gt[:, 0].detach()

    frames = sample[('color', 0, 0)]
    disp_gt = sample["disp_gt"]
    disp_gt = (height // 2) / disp_gt.to(torch.float32)
    disp_gt = torch.clamp(disp_gt, min_depth, max_depth)
    depth_gt_sample = disp_gt[:, 0].detach()

    # Store ground truth depth maps
    depth_gt_list.append(depth_gt_sample.cpu())

    # Collect frames for visualization
    fig, (ax1, ax2) = plt.subplots(figsize=(10, 10), nrows=1, ncols=2)
    frame_data = frames
    im_8 = ax1.imshow(frame_data[0].permute(1, 2, 0).cpu().detach().numpy())
    im_9 = ax2.imshow(depth_gt_sample.permute(1, 2, 0).cpu().numpy(), cmap='viridis_r')
    list_frames_traj_8.append(im_8)
    list_gt_frames_traj_8.append(im_9)
    plt.close(fig)

    # Get IMU parameters
    next_imu = sample[('preint_imu', -1, 0)]
    alpha.append(next_imu["alpha"])
    R_c.append(next_imu["R_c"])
    R_cbt_bc.append(next_imu["R_cbt_bc"])
    delta_t.append(next_imu["delta_t"])
    avg_velocity.append(next_imu["v_norm"])

# Compute average velocity
vel_8 = torch.mean(torch.stack(avg_velocity), dim=0)
print(f"Average velocity: {vel_8}")

# Add code to create video of the original trajectory
image_frames_traj_8 = [im.get_array() for im in list_frames_traj_8]
gt_frames_traj_8 = [im.get_array() for im in list_gt_frames_traj_8]

# Create a figure and axis for the animation
fig, ax = plt.subplots()
# Initialize the image on the axis (using the first frame's shape)
im = ax.imshow(image_frames_traj_8[0], cmap='viridis')

# Update function for animation
def update(frame):
    im.set_data(frame)
    return [im]

# Create an animation
ani = animation.FuncAnimation(
    fig, update, frames=image_frames_traj_8, interval=100, blit=True
)

# Save the animation as a video file (e.g., MP4)
ani.save('Trajectories_anim/trajectory_8.mp4', writer='ffmpeg', fps=25)
plt.close(fig)

# Depth prediction using the same method as the second file
TRANS_SCALE_FACTOR = 5.4
pair_inputs = [init_sample[('color', 0, 0)], init_sample[('color', 1, 0)]]

# Generate velocity and gravity inputs
velo_inputs = [models["velo_encoder"](torch.cat(pair_inputs, 1))]
gravity_inputs = [models["gravity_encoder"](torch.cat(pair_inputs, 1))]

velocity = models["velo"](velo_inputs)
gravity = models["gravity"](gravity_inputs)

R_cbt_bc[0] = R_cbt_bc[0]

# Get depth prediction
with torch.no_grad():
    output = models["depth"](models["encoder"](init_image))
    disp_trajectory_8 = output

# Adjusted depth prediction
pred_disp, _ = disp_to_depth(disp_trajectory_8[("disp", 0)], min_depth, max_depth)
pred_disp = pred_disp.cpu()[:, 0]
pred_depth = 1 / pred_disp

# Apply median scaling
ratio = np.median(depth_gt.cpu().numpy()) / np.median(pred_depth.cpu().numpy())
pred_depth *= ratio
pred_depth = pred_depth.to(init_image.device).unsqueeze(0)

# Ensure depth values are within valid range
pred_depth = torch.clamp(pred_depth, min_depth, max_depth)

# Generate poses and depth trajectory
poses_depth = get_poses_depth(dataset_trajectory_8, models, alpha, R_c, R_cbt_bc, delta_t, TRANS_SCALE_FACTOR)
depth_after_imu, depth_init_trajectory_8, images_after_imu = get_depth_trajectory(
    batch_size, init_image, disp_trajectory_8[("disp", 0)], poses_depth, init_sample
)

# Save depth video
get_video_depth(depth_after_imu, "Depth_anim/depth_trajectory_8.mp4")

# Function to compute errors
def compute_errors(gt, pred):
    """Compute error metrics between predicted and ground truth depths."""
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25     ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse_trajectory_8 = np.sqrt(((gt - pred) ** 2).mean())
    rmse_log = np.sqrt(((np.log(gt) - np.log(pred)) ** 2).mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)
    sq_rel = np.mean(((gt - pred) ** 2) / gt)

    return abs_rel, sq_rel, rmse_trajectory_8, rmse_log, a1, a2, a3

# Now, compute errors between the predicted depth maps and ground truth
# Collect predicted depth maps from depth_after_imu
predicted_depth_maps = [depth.cpu().squeeze(0) for depth in depth_after_imu]

errors_trajectory_8 = []

for pred_depth_map, gt_depth_map in zip(predicted_depth_maps, depth_gt_list):
    pred_depth_map_np = pred_depth_map.detach().numpy() 
    gt_depth_map_np = gt_depth_map.numpy()
    
    # Apply mask to valid depth values
    mask = (gt_depth_map_np > min_depth) & (gt_depth_map_np < max_depth)
    
    pred_depth_flat = pred_depth_map_np[mask]
    gt_depth_flat = gt_depth_map_np[mask]
    
    # Ensure predicted depths are within valid range
    pred_depth_flat = np.clip(pred_depth_flat, min_depth, max_depth)
    
    # Compute error metrics
    errors_trajectory_8.append(compute_errors(gt_depth_flat, pred_depth_flat))

# Compute mean errors
mean_errors = np.mean(errors_trajectory_8, axis=0)

print("\n  " + ("{:>8} | " * 7).format("abs_rel", "sq_rel", "rmse_trajectory_8", "rmse_log", "a1", "a2", "a3"))
print(("{: 8.3f}  " * 7).format(*mean_errors))


In [ ]:
rmse_trajectory_8 = [item[2] for item in errors_trajectory_8]
plt.title("RMSE as prediction horizon increases")
plt.plot(rmse_trajectory_8)
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")

In [ ]:
np.save('rmse_trajectories/rmse_trajectory_8.npy', rmse_trajectory_8)

In [ ]:
np.save("velocities/vel_8.npy", vel_8)

#### Trajectory 9

In [ ]:
print(os.path.join("splits", "eigen_midair", "{}_files.txt"))
fpath = os.path.join("splits","eigen_midair", "{}_files.txt")

train_filenames = readlines(fpath.format("train"))

train_filenames = sorted(train_filenames, key=custom_sort_key)

In [ ]:
# Main code
# INIT DEPTH PREDICTION
min_depth = 0.1
max_depth = 80

height = 512
width = 512
batch_size = 1

backproject_depth = BackprojectDepth(batch_size, height, width)
project_3d = Project3D(batch_size, height, width)
fraction_trajectory_9 = train_filenames[24248:24248+100]

dataset = datasets.midair.MidAirDataset
fpath = os.path.join("splits", "eigen_midair", "{}_files.txt")
data_path = "MidAir"

print("Creating dataset trajectory 1")
dataset_trajectory_9 = dataset(
    data_path, fraction_trajectory_9, 
    height, width,
    [0, -1, 1],
    select_file_names=True,
    num_scales=1, 
    use_imu=True,
    k_imu_clip=5, is_train=False, img_ext='.jpg' 
)   

In [ ]:
# Initialize lists to store data
dataloader_trajectory_9 = DataLoader(dataset_trajectory_9, 1, shuffle=False, num_workers=0, pin_memory=True, drop_last=False)
list_frames_traj_9 = []
list_gt_frames_traj_9 = []
depth_gt_list = []  # List to store ground truth depth maps
alpha, R_c, R_cbt_bc, delta_t, gravities, velocities = [], [], [], [], [], []

avg_velocity = []

for idx, sample in enumerate(dataset_trajectory_9):
    # Standard processing
    for key, ipt in sample.items():
        if key in [("preint_imu", -1, 0), ("preint_imu", 0, 1)]:
            for pkey, pipt in sample[key].items():
                sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                if pkey not in ["v_norm", "delta_t"]:
                    pipt = pipt.unsqueeze(0)
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
                else:
                    sample[key][pkey] = pipt.to(device, dtype=torch.float32)
        else:
            ipt = ipt.unsqueeze(0)
            sample[key] = ipt.to(device, dtype=torch.float32)

    if idx == 0:
        init_sample = sample
        init_image = sample[('color', 0, 0)]
        depth_gt = sample["disp_gt"]
        depth_gt = (height // 2) / depth_gt.to(torch.float32)
        depth_gt = torch.clamp(depth_gt, min_depth, max_depth)
        depth_gt = depth_gt[:, 0].detach()

    frames = sample[('color', 0, 0)]
    disp_gt = sample["disp_gt"]
    disp_gt = (height // 2) / disp_gt.to(torch.float32)
    disp_gt = torch.clamp(disp_gt, min_depth, max_depth)
    depth_gt_sample = disp_gt[:, 0].detach()

    # Store ground truth depth maps
    depth_gt_list.append(depth_gt_sample.cpu())

    # Collect frames for visualization
    fig, (ax1, ax2) = plt.subplots(figsize=(10, 10), nrows=1, ncols=2)
    frame_data = frames
    im_9 = ax1.imshow(frame_data[0].permute(1, 2, 0).cpu().detach().numpy())
    im_10 = ax2.imshow(depth_gt_sample.permute(1, 2, 0).cpu().numpy(), cmap='viridis_r')
    list_frames_traj_9.append(im_9)
    list_gt_frames_traj_9.append(im_10)
    plt.close(fig)

    # Get IMU parameters
    next_imu = sample[('preint_imu', -1, 0)]
    alpha.append(next_imu["alpha"])
    R_c.append(next_imu["R_c"])
    R_cbt_bc.append(next_imu["R_cbt_bc"])
    delta_t.append(next_imu["delta_t"])
    avg_velocity.append(next_imu["v_norm"])

# Compute average velocity
vel_9 = torch.mean(torch.stack(avg_velocity), dim=0)
print(f"Average velocity: {vel_9}")

# Add code to create video of the original trajectory
image_frames_traj_9 = [im.get_array() for im in list_frames_traj_9]
gt_frames_traj_9 = [im.get_array() for im in list_gt_frames_traj_9]

# Create a figure and axis for the animation
fig, ax = plt.subplots()
# Initialize the image on the axis (using the first frame's shape)
im = ax.imshow(image_frames_traj_9[0], cmap='viridis')

# Update function for animation
def update(frame):
    im.set_data(frame)
    return [im]

# Create an animation
ani = animation.FuncAnimation(
    fig, update, frames=image_frames_traj_9, interval=100, blit=True
)

# Save the animation as a video file (e.g., MP4)
ani.save('Trajectories_anim/trajectory_9.mp4', writer='ffmpeg', fps=25)
plt.close(fig)

# Depth prediction using the same method as the second file
TRANS_SCALE_FACTOR = 5.4
pair_inputs = [init_sample[('color', 0, 0)], init_sample[('color', 1, 0)]]

# Generate velocity and gravity inputs
velo_inputs = [models["velo_encoder"](torch.cat(pair_inputs, 1))]
gravity_inputs = [models["gravity_encoder"](torch.cat(pair_inputs, 1))]

velocity = models["velo"](velo_inputs)
gravity = models["gravity"](gravity_inputs)

R_cbt_bc[0] = R_cbt_bc[0]

# Get depth prediction
with torch.no_grad():
    output = models["depth"](models["encoder"](init_image))
    disp_trajectory_9 = output

# Adjusted depth prediction
pred_disp, _ = disp_to_depth(disp_trajectory_9[("disp", 0)], min_depth, max_depth)
pred_disp = pred_disp.cpu()[:, 0]
pred_depth = 1 / pred_disp

# Apply median scaling
ratio = np.median(depth_gt.cpu().numpy()) / np.median(pred_depth.cpu().numpy())
pred_depth *= ratio
pred_depth = pred_depth.to(init_image.device).unsqueeze(0)

# Ensure depth values are within valid range
pred_depth = torch.clamp(pred_depth, min_depth, max_depth)

# Generate poses and depth trajectory
poses_depth = get_poses_depth(dataset_trajectory_9, models, alpha, R_c, R_cbt_bc, delta_t, TRANS_SCALE_FACTOR)
depth_after_imu, depth_init_trajectory_9, images_after_imu = get_depth_trajectory(
    batch_size, init_image, disp_trajectory_9[("disp", 0)], poses_depth, init_sample
)

# Save depth video
get_video_depth(depth_after_imu, "Depth_anim/depth_trajectory_9.mp4")

# Function to compute errors
def compute_errors(gt, pred):
    """Compute error metrics between predicted and ground truth depths."""
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25     ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse_trajectory_9 = np.sqrt(((gt - pred) ** 2).mean())
    rmse_log = np.sqrt(((np.log(gt) - np.log(pred)) ** 2).mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)
    sq_rel = np.mean(((gt - pred) ** 2) / gt)

    return abs_rel, sq_rel, rmse_trajectory_9, rmse_log, a1, a2, a3

# Now, compute errors between the predicted depth maps and ground truth
# Collect predicted depth maps from depth_after_imu
predicted_depth_maps = [depth.cpu().squeeze(0) for depth in depth_after_imu]

errors_trajectory_9 = []

for pred_depth_map, gt_depth_map in zip(predicted_depth_maps, depth_gt_list):
    pred_depth_map_np = pred_depth_map.detach().numpy() 
    gt_depth_map_np = gt_depth_map.numpy()
    
    # Apply mask to valid depth values
    mask = (gt_depth_map_np > min_depth) & (gt_depth_map_np < max_depth)
    
    pred_depth_flat = pred_depth_map_np[mask]
    gt_depth_flat = gt_depth_map_np[mask]
    
    # Ensure predicted depths are within valid range
    pred_depth_flat = np.clip(pred_depth_flat, min_depth, max_depth)
    
    # Compute error metrics
    errors_trajectory_9.append(compute_errors(gt_depth_flat, pred_depth_flat))

# Compute mean errors
mean_errors = np.mean(errors_trajectory_9, axis=0)

print("\n  " + ("{:>8} | " * 7).format("abs_rel", "sq_rel", "rmse_trajectory_9", "rmse_log", "a1", "a2", "a3"))
print(("{: 8.3f}  " * 7).format(*mean_errors))


In [ ]:
rmse_trajectory_9 = [item[2] for item in errors_trajectory_9]
plt.title("RMSE as prediction horizon increases")
plt.plot(rmse_trajectory_9)
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")

In [ ]:
np.save('rmse_trajectories/rmse_trajectory_9.npy', rmse_trajectory_9)

In [ ]:
np.save("velocities/vel_9.npy", vel_9)

In [ ]:
np.save("velocities/vel_8.npy", vel_8)

#### Plot trajectories in same frame

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Load the stored RMSE values for different trajectories (for the example, I'm loading the same file multiple times)
rmse_trajectory_files = ["/Users/ibrahimhassan/Documents/Documents/thesis_DynaDepth/rmse_trajectories/rmse_trajectory_2.npy",
                         "/Users/ibrahimhassan/Documents/Documents/thesis_DynaDepth/rmse_trajectories/rmse_trajectory_3.npy",
                         "/Users/ibrahimhassan/Documents/Documents/thesis_DynaDepth/rmse_trajectories/rmse_trajectory_6.npy",
                         "/Users/ibrahimhassan/Documents/Documents/thesis_DynaDepth/rmse_trajectories/rmse_trajectory_7.npy",
                         "/Users/ibrahimhassan/Documents/Documents/thesis_DynaDepth/rmse_trajectories/rmse_trajectory_8.npy"]  # Example file paths; add others when available


velocities_list = ["/Users/ibrahimhassan/Documents/Documents/thesis_DynaDepth/velocities/vel_2.npy",
                   "/Users/ibrahimhassan/Documents/Documents/thesis_DynaDepth/velocities/vel_3.npy",
                   "/Users/ibrahimhassan/Documents/Documents/thesis_DynaDepth/velocities/vel_6.npy",
                   "/Users/ibrahimhassan/Documents/Documents/thesis_DynaDepth/velocities/vel_7.npy",
                   "/Users/ibrahimhassan/Documents/Documents/thesis_DynaDepth/velocities/vel_8.npy"]
# Dictionary to hold RMSE values for each trajectory
rmse_trajectories = {}

# Load each RMSE file and store in the dictionary
for idx, file_path in enumerate(rmse_trajectory_files, start=1):
    rmse_trajectories[f"Trajectory_{idx}"] = np.load(file_path)

# Plotting multiple trajectories in the same frame
plt.figure(figsize=(10, 6))
for trajectory_name, rmse_values in rmse_trajectories.items():
    plt.plot(rmse_values, label=trajectory_name)

plt.title("RMSE as Prediction Horizon Increases for Multiple Trajectories")
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")
plt.legend()
plt.show()


#### 30 trajectories simulation

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import gc  # Import garbage collector


# Assuming 'readlines' and 'custom_sort_key' functions are defined
print(os.path.join("splits", "eigen_midair", "{}_files.txt"))
fpath = os.path.join("splits","eigen_midair", "{}_files.txt")

train_filenames = readlines(fpath.format("train"))

train_filenames = sorted(train_filenames, key=custom_sort_key)


In [ ]:
# INIT DEPTH PREDICTION
min_depth = 0.1
max_depth = 80

height = 512
width = 512
batch_size = 1

backproject_depth = BackprojectDepth(batch_size, height, width)
project_3d = Project3D(batch_size, height, width)

dataset = datasets.midair.MidAirDataset
data_path = "MidAir"


In [ ]:
def compute_errors(gt, pred):
    """Compute error metrics between predicted and ground truth depths."""
    thresh = np.maximum((gt / pred), (pred / gt))
    a1 = (thresh < 1.25     ).mean()
    a2 = (thresh < 1.25 ** 2).mean()
    a3 = (thresh < 1.25 ** 3).mean()

    rmse = np.sqrt(((gt - pred) ** 2).mean())
    rmse_log = np.sqrt(((np.log(gt + 1e-6) - np.log(pred + 1e-6)) ** 2).mean())

    abs_rel = np.mean(np.abs(gt - pred) / (gt + 1e-6))
    sq_rel = np.mean(((gt - pred) ** 2) / (gt + 1e-6))

    return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3


In [ ]:
num_trajectories = 30
trajectory_length = 100

# Ensure there are enough filenames
assert len(train_filenames) >= num_trajectories * trajectory_length, "Not enough filenames to create trajectories."

all_rmse_trajectories = {}
all_velocities = {}

for traj_idx in range(0,num_trajectories):
    print(f"idx: {traj_idx}/{num_trajectories}")
    # Calculate start and end indices for the trajectory in train_filenames
    start_idx = traj_idx * trajectory_length
    end_idx = start_idx + trajectory_length
    
    fraction_trajectory = train_filenames[start_idx:end_idx]
    
    # Create dataset for the trajectory
    dataset_trajectory = dataset(
        data_path, fraction_trajectory, 
        height, width,
        [0, -1, 1],
        select_file_names=True,
        num_scales=1, 
        use_imu=True,
        k_imu_clip=5, is_train=False, img_ext='.jpg'
    )
    
    # Initialize lists to store data for this trajectory
    depth_gt_list = []  # List to store ground truth depth maps
    alpha, R_c, R_cbt_bc, delta_t, gravities, velocities = [], [], [], [], [], []
    avg_velocity = []
    errors_trajectory = []
    
    # Data loader for the trajectory
    dataloader_trajectory = DataLoader(dataset_trajectory, 1, shuffle=False, num_workers=0, pin_memory=True, drop_last=False)
    
    for idx, sample in enumerate(dataset_trajectory):
        # Standard processing
        for key, ipt in sample.items():
            if key in [("preint_imu", -1, 0), ("preint_imu", 0, 1)]:
                for pkey, pipt in sample[key].items():
                    pipt = pipt.to(device, dtype=torch.float32)
                    if pkey not in ["v_norm", "delta_t"]:
                        pipt = pipt.unsqueeze(0)
                    sample[key][pkey] = pipt
            else:
                ipt = ipt.unsqueeze(0).to(device, dtype=torch.float32)
                sample[key] = ipt
        
        if idx == 0:
            init_sample = sample
            init_image = sample[('color', 0, 0)]
            disp_gt = sample["disp_gt"]
            depth_gt_init = (height // 2) / disp_gt.to(torch.float32)
            depth_gt_init = torch.clamp(depth_gt_init, min_depth, max_depth)
            depth_gt_init = depth_gt_init[:, 0].detach()
        
        frames = sample[('color', 0, 0)]
        disp_gt = sample["disp_gt"]
        disp_gt = (height // 2) / disp_gt.to(torch.float32)
        disp_gt = torch.clamp(disp_gt, min_depth, max_depth)
        depth_gt_sample = disp_gt[:, 0].detach()
        
        # Store ground truth depth maps
        depth_gt_list.append(depth_gt_sample.cpu())
        
        # Get IMU parameters
        next_imu = sample[('preint_imu', -1, 0)]
        alpha.append(next_imu["alpha"])
        R_c.append(next_imu["R_c"])
        R_cbt_bc.append(next_imu["R_cbt_bc"])
        delta_t.append(next_imu["delta_t"])
        avg_velocity.append(next_imu["v_norm"])
    
    # Compute average velocity for this trajectory
    vel = torch.mean(torch.stack(avg_velocity), dim=0)
    print(f"Average velocity for trajectory {traj_idx+1}: {vel.item()}")
    all_velocities[f"Trajectory_{traj_idx+1}"] = vel.item()
    
    # Depth prediction using the same method as before
    TRANS_SCALE_FACTOR = 5.4
    pair_inputs = [init_sample[('color', 0, 0)], init_sample[('color', 1, 0)]]
    
    # Generate velocity and gravity inputs
    velo_inputs = [models["velo_encoder"](torch.cat(pair_inputs, 1))]
    gravity_inputs = [models["gravity_encoder"](torch.cat(pair_inputs, 1))]
    
    velocity_pred = models["velo"](velo_inputs)
    gravity = models["gravity"](gravity_inputs)
    
    R_cbt_bc[0] = R_cbt_bc[0]
    
    # Get depth prediction
    with torch.no_grad():
        output = models["depth"](models["encoder"](init_image))
        disp_trajectory = output
    
    # Adjusted depth prediction
    pred_disp, _ = disp_to_depth(disp_trajectory[("disp", 0)], min_depth, max_depth)
    pred_disp = pred_disp.cpu()[:, 0]
    pred_depth = 1 / pred_disp
    
    # Apply median scaling
    ratio = np.median(depth_gt_init.cpu().numpy()) / np.median(pred_depth.cpu().numpy())
    pred_depth *= ratio
    pred_depth = pred_depth.to(init_image.device).unsqueeze(0)
    
    # Compute errors between predicted and ground truth depths
    ##########################################################
    pred_depth_tmp = pred_depth.squeeze()
    pred_depth_tmp = pred_depth_tmp.detach().cpu().numpy()
    depth_gt_init_tmp = depth_gt_init.squeeze()
    depth_gt_init_tmp = depth_gt_init_tmp.detach().cpu().numpy()

    mask = (depth_gt_init_tmp > min_depth) & (depth_gt_init_tmp < max_depth)
    pred_depth_flat = pred_depth_tmp[mask]

    gt_depth_flat = depth_gt_init_tmp[mask]
    print(f"init error : {compute_errors(gt_depth_flat, pred_depth_flat)[2]}")
    ##########################################################
    
    # Generate poses and depth trajectory
    poses_depth = get_poses_depth(dataset_trajectory, models, alpha, R_c, R_cbt_bc, delta_t, TRANS_SCALE_FACTOR)
    depth_after_imu, depth_init_trajectory, images_after_imu = get_depth_trajectory(
        batch_size, init_image, pred_depth, poses_depth, init_sample
    )
    
    # Now, compute errors between the predicted depth maps and ground truth
    # Collect predicted depth maps from depth_after_imu
    predicted_depth_maps = [depth.cpu().squeeze(0) for depth in depth_after_imu]
    
    for pred_depth_map, gt_depth_map in zip(predicted_depth_maps, depth_gt_list):
        pred_depth_map_np = pred_depth_map.detach().numpy() 
        gt_depth_map_np = gt_depth_map.numpy()
        
        # Apply mask to valid depth values
        mask = (gt_depth_map_np > min_depth) & (gt_depth_map_np < max_depth)
        
        pred_depth_flat = pred_depth_map_np[mask]
        gt_depth_flat = gt_depth_map_np[mask]
        
        # Ensure predicted depths are within valid range
        pred_depth_flat = np.clip(pred_depth_flat, min_depth, max_depth)
        
        # Compute error metrics
        errors = compute_errors(gt_depth_flat, pred_depth_flat)
        errors_trajectory.append(errors)
    
    # Extract RMSE values for this trajectory
    rmse_values = [item[2] for item in errors_trajectory]
    all_rmse_trajectories[f"Trajectory_{traj_idx+1}"] = rmse_values

    # Optionally, save RMSE values for this trajectory
    np.save(f"rmse_values_trajectory_{traj_idx+1}.npy", rmse_values)

    # Memory Management: Release unnecessary variables and clear cache
    del dataset_trajectory
    del dataloader_trajectory
    del init_sample, init_image, depth_gt_init, disp_gt
    del frames
    del alpha, R_c, R_cbt_bc, delta_t, avg_velocity
    del velo_inputs, gravity_inputs, velocity_pred, gravity
    del output, disp_trajectory, pred_disp, pred_depth
    del poses_depth, depth_after_imu, depth_init_trajectory, images_after_imu
    del predicted_depth_maps, depth_gt_list
    gc.collect()


In [ ]:
import os
import numpy as np

# Define the folder path
folder_path = 'rmse_traj'

# Initialize the dictionary to store RMSE values
all_rmse_trajectories = {}

# Iterate over the files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.npy'):
        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)
        
        # Load the RMSE values from the file
        rmse_values = np.load(file_path)
        
        # Extract the trajectory number from the file name
        trajectory_number = file_name.split('_')[-1].split('.')[0]
        
        # Store the RMSE values in the dictionary
        all_rmse_trajectories[f'Trajectory_{trajectory_number}'] = rmse_values

# Print the dictionary to verify
print(sorted(all_rmse_trajectories.keys()))

In [ ]:
filtered_rmse_trajectories = {key:all_rmse_trajectories[key] for key in all_rmse_trajectories.keys() if all_rmse_trajectories[key][0] <20 and all(value < 25 for value in all_rmse_trajectories[key][:10])}

In [ ]:
plt.figure(figsize=(12, 8))

for trajectory_name, rmse_values in all_rmse_trajectories.items():
    plt.plot(rmse_values, label=trajectory_name)

plt.title("RMSE as Prediction Horizon Increases for Multiple Trajectories")
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")
# plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

for trajectory_name, rmse_values in filtered_rmse_trajectories.items():
    plt.plot(rmse_values, color='lightgrey', linewidth=1, label=trajectory_name)

# plt.title("RMSE as Prediction Horizon Increases for Multiple Trajectories")
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")
# plt.legend()
plt.show()

In [ ]:
filtered_rmse_trajectories_vel_1 = {key:all_rmse_trajectories[key] for key in all_rmse_trajectories.keys() if all_rmse_trajectories[key][0] <20 and all_velocities[key] >5  and all(value < 25 for value in all_rmse_trajectories[key][:10])}

In [ ]:
plt.figure(figsize=(12, 8))

for trajectory_name, rmse_values in filtered_rmse_trajectories_vel_1.items():
    plt.plot(rmse_values, color='lightgrey', linewidth=1, label=trajectory_name)

plt.title("RMSE as Prediction Horizon Increases for Multiple Trajectories")
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")
# plt.legend()
plt.show()

In [ ]:
# Calculate the average RMSE excluding the three largest trajectories based on their final RMSE values.
sorted_trajectories = sorted(filtered_rmse_trajectories.items(), key=lambda x: x[1][-1])

# Exclude the three largest trajectories.
filtered_values = [values for _, values in sorted_trajectories[:-3]]

# Calculate the mean RMSE at each time step across the remaining trajectories.
average_rmse = np.mean(filtered_values, axis=0)

# Plotting the RMSE values for each trajectory in light grey
plt.figure(figsize=(12, 8))

for trajectory_name, rmse_values in filtered_rmse_trajectories.items():
    plt.plot(rmse_values, color='lightgrey', linewidth=1)

# Adding the average line in a bold color
plt.plot(average_rmse, color='blue', linewidth=3, label='Average RMSE (Excluding 3 Largest)')

plt.title("RMSE as Prediction Horizon Increases for Multiple Trajectories")
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")
plt.legend()
plt.show()

In [ ]:
filtered_rmse_trajectories_vel_1 = {key:all_rmse_trajectories[key] for key in all_rmse_trajectories.keys() if all_rmse_trajectories[key][0] <10}

In [ ]:
filtered_rmse_trajectories_vel_1.keys()

#### PLot of trajectories with initial error smaller than 10

In [ ]:
# Plotting the RMSE values for each trajectory in light grey
plt.figure(figsize=(12, 8))

for trajectory_name, rmse_values in filtered_rmse_trajectories_vel_1.items():
    plt.plot(rmse_values, color='lightgrey', linewidth=1)

plt.title("RMSE as Prediction Horizon Increases for Multiple Trajectories (Excluding 3 Largest)")
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")
plt.show()


In [ ]:
first_error = [all_rmse_trajectories[key][0] for key in all_rmse_trajectories.keys() if all_rmse_trajectories[key][0] <10]

In [ ]:
# Calculate the average RMSE excluding the three largest trajectories based on their final RMSE values.
sorted_trajectories = sorted(filtered_rmse_trajectories_vel_1.items(), key=lambda x: x[1][-1])

# Exclude the three largest trajectories.
filtered_values = [values for _, values in sorted_trajectories[:-3]]

# Calculate the mean RMSE at each time step across the remaining trajectories.
average_rmse = np.mean(filtered_values, axis=0)

# Plotting the RMSE values for each trajectory in light grey
plt.figure(figsize=(12, 8))

for trajectory_name, rmse_values in filtered_rmse_trajectories_vel_1.items():
    plt.plot(rmse_values, color='lightgrey', linewidth=1)

# Adding the average line in a bold color
plt.plot(average_rmse, color='blue', linewidth=3, label='Average RMSE (Excluding 3 Largest)')
plt.grid(visible=True, linestyle='--', alpha=0.6)

plt.title("RMSE as Prediction Horizon Increases for Multiple Trajectories")
plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")
plt.legend()
plt.show()

In [ ]:
all_rmse_trajectories

In [ ]:
filtered_rmse_trajectories_fil = {key:all_rmse_trajectories[key] for key in all_rmse_trajectories.keys() if all_rmse_trajectories[key][0] <10 and all(value < 28 for value in all_rmse_trajectories[key])}

In [ ]:
filtered_rmse_trajectories_slope = {}
for key, rmse_values in filtered_rmse_trajectories_fil.items():
    # Calculate average slope over the first 10 values
    slopes = np.diff(rmse_values[:10])
    average_slope = np.mean(np.abs(slopes))  # take absolute values for positive slope
    
    # Keep only trajectories with average slope below the threshold
    if abs(average_slope) <0.2:
        filtered_rmse_trajectories_slope[key] = rmse_values

In [ ]:
filtered_rmse_trajectories_slope
count = sum(1 for key, value in filtered_rmse_trajectories_slope.items() if value[0] < 10)
print(count)
mean_value = np.mean([value[0] for key, value in filtered_rmse_trajectories_slope.items() if value[0] < 10])
var_value = np.std([value[0] for key, value in filtered_rmse_trajectories_slope.items() if value[0] < 10])
print(mean_value)
print(var_value)

In [ ]:
# filtered_rmse_trajectories_vel_1 = {key:all_rmse_trajectories[key] for key in all_rmse_trajectories.keys() if all_rmse_trajectories[key][0] <10}
# Calculate the average RMSE excluding the three largest trajectories based on their final RMSE values.
sorted_trajectories = sorted(filtered_rmse_trajectories_slope.items(), key=lambda x: x[1][-1])

# Exclude the three largest trajectories.
filtered_values = [values for _, values in sorted_trajectories if values[0] < 10]



# Calculate the mean RMSE at each time step across the remaining trajectories.
average_rmse = np.mean(filtered_values, axis=0)


plt.figure(figsize=(12, 8))

for trajectory_name, rmse_values in filtered_rmse_trajectories_slope.items():
    plt.plot(rmse_values, color='lightgrey', linewidth=1, label=trajectory_name)

plt.plot(average_rmse, color='blue', linewidth=3, label='Average RMSE (Excluding 3 Largest)')


# plt.title("RMSE as Prediction Horizon Increases for Multiple Trajectories")
plt.grid(visible=True, linestyle='--', alpha=0.6)

plt.xlabel("Prediction Horizon")
plt.ylabel("RMSE")
plt.xlim(0,50)
plt.xticks(np.arange(0, 51, step=5))
# plt.legend()
plt.show()